ARIMA model (including tuning)

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", message="Non-invertible starting MA parameters found. Using zeros as starting parameters.")
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from pmdarima.arima import auto_arima
from joblib import Parallel, delayed
from datetime import timedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from concurrent.futures import ProcessPoolExecutor, as_completed
import itertools
import ast
import time


def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['start_datetime', 'end_datetime'])

    train_df = df[(df['start_datetime'].dt.year == 2023)]
    val_df = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
    test_df  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]
    newtrain_df = pd.concat([train_df, val_df], ignore_index=True)

    train_df = train_df.copy()
    val_df = val_df.copy()
    test_df = test_df.copy()
    newtrain_df = newtrain_df.copy()

    for lag in [24, 48, 72, 168]:
        train_df[f'price_lag_{lag}'] = train_df['Price'].shift(lag)
        val_df[f'price_lag_{lag}'] = val_df['Price'].shift(lag)
        test_df[f'price_lag_{lag}'] = test_df['Price'].shift(lag)
        newtrain_df[f'price_lag_{lag}'] = newtrain_df['Price'].shift(lag)

    train_df.dropna(inplace=True)
    val_df.dropna(inplace=True)
    test_df.dropna(inplace=True)
    newtrain_df.dropna(inplace=True)

    return train_df, val_df, test_df, newtrain_df

def fit_hourly_arima_models_24(train_df, order):
    models = {}
    for hour in tqdm(range(24), desc="Fitting ARIMA models for each hour of the day"):
        df_filtered = train_df[train_df['start_datetime'].dt.hour == hour].copy()
        df_filtered = df_filtered.set_index('start_datetime').asfreq('D')
        df_filtered.dropna(inplace=True)

        model = ARIMA(df_filtered['Price'], order=order, enforce_stationarity=False, enforce_invertibility=False).fit()
        models[hour] = model
    return models

def forecast_next_24(test_df, test_days, models, Forecast_horizon):
    steps_ahead = Forecast_horizon // 24 
    test_df = test_df.copy()
    test_df['date'] = test_df['start_datetime'].dt.normalize()
    test_df['hour'] = test_df['start_datetime'].dt.hour

    forecasts, true_vals, start_times = [], [], []
    actual_by_hour = {h: test_df[test_df['hour'] == h].set_index('start_datetime')['Price'] for h in range(24)}

    for d in tqdm(test_days, desc=f"ARIMA Forecasting {Forecast_horizon}h with 24 hourly models"):
        forecast = np.empty(Forecast_horizon)
        forecast[:] = np.nan

        day_to_append_start = d
        day_to_append_end = d + timedelta(days=1)

        for hour in range(24):
            model_fit = models[hour]

            forecast_vals = model_fit.forecast(steps=steps_ahead)

            for step in range(steps_ahead):
                idx = step * 24 + hour
                if idx < Forecast_horizon:
                    forecast[idx] = forecast_vals.iloc[step]

    
            actuals_for_day_d = actual_by_hour[hour].loc[
                (actual_by_hour[hour].index >= day_to_append_start) &
                (actual_by_hour[hour].index < day_to_append_end)
            ]

            if not actuals_for_day_d.empty:
                model_fit = model_fit.append(actuals_for_day_d.values)
                models[hour] = model_fit
    
        d_start = d
        d_end = d + timedelta(hours = Forecast_horizon-1)
        mask = (test_df['start_datetime'] >= d_start) & (test_df['start_datetime'] <= d_end)
        actual_prices = test_df[mask]['Price'].values

        if len(actual_prices) == Forecast_horizon:
            forecasts.append(forecast)
            true_vals.append(actual_prices)
            start_times.append(d)
        else:
            print(f"Warning: Mismatch in forecast length for day {d}. Expected {Forecast_horizon}, got {len(actual_prices)}")

    return np.array(forecasts), np.array(true_vals), start_times


train_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/train_dates.csv', parse_dates=['date']).sort_values(by='date')
val_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/val_dates.csv', parse_dates=['date']).sort_values(by='date')
test_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/test_dates.csv', parse_dates=['date']).sort_values(by='date')
train_dates = train_dates['date'].tolist()
val_dates = val_dates['date'].tolist()
test_dates = test_dates['date'].tolist()

FILE_PATH = 'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv'
train_df, val_df, test_df, newtrain_df = load_data(FILE_PATH)

def tune_arima():
    results = []
    for forecast_horizon in [48, 144]:
        start_time = time.time()
        print(f"\nTuning ARIMA for forecast horizon: {forecast_horizon} hours")
        p_values = [1, 2, 3, 4]
        d_values = [1]
        q_values = [1, 2, 3, 4]

        best_rmse = float("inf")
        best_order = None

        i = 0
        total_models = len(p_values) * len(d_values) * len(q_values)
        for p in p_values:
            for d in d_values:
                for q in q_values:
                    i += 1
                    try:
                        order = (p, d, q)
                        print(f"\nTrying ARIMA({p},{d},{q}). Model {i} of {total_models}")
                        models = fit_hourly_arima_models_24(train_df, order)
                        forecasts, true_vals, _ = forecast_next_24(val_df, val_dates, models, forecast_horizon)
                        rmse = np.sqrt(mean_squared_error(true_vals, forecasts))
                        print(f"Validation RMSE: {rmse:.2f}")
            
                        if rmse < best_rmse:
                            best_rmse = rmse
                            best_order = (p, d, q)
                            print(f"New best ARIMA order: {best_order} RMSE: {best_rmse:.2f}")
                    except Exception as e:
                        print(f"ARIMA({p},{d},{q}) failed: {e}")

        print(f"\n✅ Best ARIMA order: {best_order} with RMSE: {best_rmse:.2f} €/MWh")
        end_time = time.time()
        run_time = end_time - start_time
        results.append((forecast_horizon, best_order, best_rmse, run_time))
    return results

#To tune the ARIMA models, uncomment the following lines:
#results = tune_arima()
#results_df = pd.DataFrame(results, columns=['Forecast Horizon', 'Best Order', 'Best RMSE', 'Run Time'])
#results_df.to_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning_new/arima_tuning_results_new.csv', index=False)


for Forecast_horizon in [48, 144]:
    tuning = pd.read_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/arima_tuning_results.csv', sep=',')
    order = tuning[tuning['Forecast Horizon'] == Forecast_horizon]['Best Order'].values[0]
    order = ast.literal_eval(order)
    print(f"Generating final forecasts for Forecast Horizon: {Forecast_horizon} and ARIMA order: {order}")
    models = fit_hourly_arima_models_24(newtrain_df, order)
    final_forecasts, final_true_vals, _ = forecast_next_24(test_df, test_dates, models, Forecast_horizon)
    test_rmse = np.sqrt(mean_squared_error(final_true_vals, final_forecasts))
    print(f"\n Final Test RMSE using best ARIMA: {test_rmse:.2f} €/MWh")
    final_forecasts_df = pd.DataFrame(final_forecasts, columns=[f'h+{i}' for i in range(Forecast_horizon)])
    final_true_vals_df = pd.DataFrame(final_true_vals, columns=[f'h+{i}' for i in range(Forecast_horizon)])
    final_forecasts_df.to_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/Arima_forecasts_{Forecast_horizon}.csv', index=False)
    final_true_vals_df.to_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Models_true/Arima_true_vals_{Forecast_horizon}.csv', index=False)



GARCH model (Including tuning)

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", message="Non-invertible starting MA parameters found. Using zeros as starting parameters.")
from arch import arch_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from pmdarima.arima import auto_arima
from joblib import Parallel, delayed
from datetime import timedelta
import ast
import time
from statsmodels.tsa.stattools import adfuller

def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['start_datetime', 'end_datetime'])

    train_df = df[(df['start_datetime'].dt.year == 2023)]
    val_df = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
    test_df  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]
    newtrain_df = pd.concat([train_df, val_df], ignore_index=True)

    train_df = train_df.copy()
    val_df = val_df.copy()
    test_df = test_df.copy()
    newtrain_df = newtrain_df.copy()

    for lag in [24, 48, 72, 168]:
        train_df[f'price_lag_{lag}'] = train_df['Price'].shift(lag)
        val_df[f'price_lag_{lag}'] = val_df['Price'].shift(lag)
        test_df[f'price_lag_{lag}'] = test_df['Price'].shift(lag)
        newtrain_df[f'price_lag_{lag}'] = newtrain_df['Price'].shift(lag)

    train_df.dropna(inplace=True)
    val_df.dropna(inplace=True)
    test_df.dropna(inplace=True)
    newtrain_df.dropna(inplace=True)

    return train_df, val_df, test_df, newtrain_df

def fit_hourly_garch_models_24(train_df, order):
    models = {}
    for hour in tqdm(range(24), desc="Fitting GARCH models for each hour of the day"):
        df_filtered = train_df[train_df['start_datetime'].dt.hour == hour].copy()
        df_filtered = df_filtered.set_index('start_datetime').asfreq('D')
        df_filtered.dropna(inplace=True)
        p, o, q = order
        #result = adfuller(df_filtered['Price'])
        #print(f"Hour {hour}: ADF p-value = {result[1]:.4f}")

        model = arch_model(df_filtered['Price'], vol='GARCH', p=p, o=o, q=q, mean='AR', lags=1, dist='normal')
        res = model.fit(disp='off')
        models[hour] = (res, df_filtered['Price'])
    return models

def forecast_next_24(test_df, test_days, models, order, Forecast_horizon):
    p, o, q = order
    steps_ahead = Forecast_horizon // 24 
    test_df = test_df.copy()
    test_df['date'] = test_df['start_datetime'].dt.normalize()
    test_df['hour'] = test_df['start_datetime'].dt.hour

    forecasts, true_vals, start_times = [], [], []
    actual_by_hour = {h: test_df[test_df['hour'] == h].set_index('start_datetime')['Price'] for h in range(24)}

    for d in tqdm(test_days, desc=f"GARCH Forecasting {Forecast_horizon}h with 24 hourly models"):
    
        forecast = np.empty(Forecast_horizon)
        forecast[:] = np.nan

        day_to_append_start = d
        day_to_append_end = d + timedelta(days=1)

        for hour in range(24):
            model_fit, data = models[hour]
            forecast_all = model_fit.forecast(horizon=steps_ahead)
            forecast_vals = forecast_all.mean.values[-1]

            for step in range(steps_ahead):
                idx = step * 24 + hour
                if idx < Forecast_horizon:
                    forecast[idx] = forecast_vals[step]

        
            actuals_for_day_d = actual_by_hour[hour].loc[
                (actual_by_hour[hour].index >= day_to_append_start) &
                (actual_by_hour[hour].index < day_to_append_end)
            ]

            if not actuals_for_day_d.empty:
                data = np.concatenate([data, actuals_for_day_d.values])
                new_model = arch_model(data, vol='GARCH', p=p, o=o, q=q, mean='AR', lags=1, dist='normal')
                model_fit = new_model.fit(disp='off')
                models[hour] = (model_fit, data)

        d_start = d
        d_end = d + timedelta(hours = Forecast_horizon-1)
        mask = (test_df['start_datetime'] >= d_start) & (test_df['start_datetime'] <= d_end)
        actual_prices = test_df[mask]['Price'].values

        if len(actual_prices) == Forecast_horizon:
            forecasts.append(forecast)
            true_vals.append(actual_prices)
            start_times.append(d)
        else:
            print(f"Warning: Mismatch in forecast length for day {d}. Expected {Forecast_horizon}, got {len(actual_prices)}")

    return np.array(forecasts), np.array(true_vals), start_times


train_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/train_dates.csv', parse_dates=['date']).sort_values(by='date')
val_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/val_dates.csv', parse_dates=['date']).sort_values(by='date')
test_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/test_dates.csv', parse_dates=['date']).sort_values(by='date')
train_dates = train_dates['date'].tolist()
val_dates = val_dates['date'].tolist()
test_dates = test_dates['date'].tolist()

FILE_PATH = 'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined.csv'
train_df, val_df, test_df, newtrain_df = load_data(FILE_PATH)

def tune_GARCH():
    results = []
    for forecast_horizon in [48, 144]:
        start_time = time.time()
        print(f"\nTuning GARCH for forecast horizon: {forecast_horizon} hours")

        p_values = [1,2,3]
        #For regular GARCH tuning, o = 0
        o_values = [1,2]
        q_values = [1, 2,3]

        best_rmse = float("inf")
        best_order = None

        i = 0
        total_models = len(p_values) * len(o_values) * len(q_values)
        for p in p_values:
            for o in o_values:
                for q in q_values:
                    i += 1
                    try:
                        order = (p, o, q)
                        print(f"\nTrying GARCH({p},{o},{q}). Model {i} of {total_models}")
                        models = fit_hourly_garch_models_24(train_df, order)
                        forecasts, true_vals, _ = forecast_next_24(val_df, val_dates, models, order, forecast_horizon)
                        rmse = np.sqrt(mean_squared_error(true_vals, forecasts))
                        print(f"Validation RMSE: {rmse:.2f}")

                        if rmse < best_rmse:
                            best_rmse = rmse
                            best_order = (p, o, q)
                            print(f"New best GARCH order: {best_order} RMSE: {best_rmse:.2f}")
                    except Exception as e:
                        print(f"GARCH({p},{o},{q}) failed: {e}")

        print(f"\n✅ Best GARCH order: {best_order} with RMSE: {best_rmse:.2f} €/MWh")
        end_time = time.time()
        run_time = end_time - start_time
        results.append((forecast_horizon, best_order, best_rmse, run_time))
    return results

# To tune the GARCH models, uncomment the following lines:
#results = tune_GARCH()
#results_df = pd.DataFrame(results, columns=['Forecast Horizon', 'Best Order', 'Best RMSE', 'Run time'])
#results_df.to_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/garch_asymmetric_tuning_results.csv', index=False)

for Forecast_horizon in [48, 144]:
    tuning = pd.read_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/garch_regular_tuning_results.csv', sep=',')
    order = tuning[tuning['Forecast Horizon'] == Forecast_horizon]['Best Order'].values[0]
    order = ast.literal_eval(order)
    print(f"Generating final forecasts for Forecast Horizon: {Forecast_horizon} and GARCH order: {order}")
    models = fit_hourly_garch_models_24(newtrain_df, order)
    final_forecasts, final_true_vals, _ = forecast_next_24(test_df, test_dates, models, order, Forecast_horizon)
    test_rmse = np.sqrt(mean_squared_error(final_true_vals, final_forecasts))
    print(f"\n Final Test RMSE using best GARCH: {test_rmse:.2f} €/MWh")
    final_forecasts_df = pd.DataFrame(final_forecasts, columns=[f'h+{i}' for i in range(Forecast_horizon)])
    final_true_vals_df = pd.DataFrame(final_true_vals, columns=[f'h+{i}' for i in range(Forecast_horizon)])
    final_forecasts_df.to_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/garch_regular_forecasts_{Forecast_horizon}.csv', index=False)
    final_true_vals_df.to_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Models_true/garch_regular_true_vals_{Forecast_horizon}.csv', index=False)



Tuning LSTM, LGBM & XGBOOST

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from datetime import timedelta

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from collections import defaultdict
from itertools import product
import time
import random

# === CONFIGURATION ===
MODEL_TYPE = "LSTM"   # "XGB" or "LGBM" or "LSTM"
H1 = 24                
H2 = 24
H3 = 120
FILE_PATH = 'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv'
OUTPUT_PATH = f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Pred_{MODEL_TYPE}_{Forecast_Horizon}.csv'
TRUTH_PATH = f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/True_{MODEL_TYPE}_{Forecast_Horizon}.csv'
load_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/load_error.csv')
Solar_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/Solar_error.csv')
WindShore_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindShore_error.csv')
WindOffShore_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindOffShore_error.csv')


def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['start_datetime', 'end_datetime'])
  
    df['hour'] = df['start_datetime'].dt.hour
    df['dayofweek'] = df['start_datetime'].dt.dayofweek
    df['month'] = df['start_datetime'].dt.month
    df['day']= df['start_datetime'].dt.date

    train_df = df[(df['start_datetime'].dt.year == 2023)]
    val_df = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
    test_df  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]

    categorical_features = ['hour', 'dayofweek', 'month', 'day', 'start_datetime', 'end_datetime', 'Timestamp']
    features_to_scale = [col for col in train_df.columns if col not in categorical_features]

    if MODEL_TYPE == 'LSTM':
                
        train_df = df[(df['start_datetime'].dt.year == 2023)]
        val_df = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
        test_df  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]
        categorical_features = ['hour', 'dayofweek', 'month', 'day', 'start_datetime', 'end_datetime', 'Timestamp']
        features_to_scale = [col for col in train_df.columns if col not in categorical_features]

        scaler = StandardScaler()
        train_scaled = pd.DataFrame(scaler.fit_transform(train_df[features_to_scale]), index=train_df.index, columns=features_to_scale)
        val_scaled   = pd.DataFrame(scaler.transform(val_df[features_to_scale]), index=val_df.index, columns=features_to_scale)
        test_scaled  = pd.DataFrame(scaler.transform(test_df[features_to_scale]), index=test_df.index, columns=features_to_scale)

        price_scaler = StandardScaler().fit(train_df[['Price']])
        for df_scaled, df_orig in zip([train_scaled, val_scaled, test_scaled], [train_df, val_df, test_df]):
            df_scaled['Price'] = price_scaler.transform(df_orig[['Price']])
            df_scaled[categorical_features] = df_orig[categorical_features]

    else:
        train_scaled = df[(df['start_datetime'].dt.year == 2023)]
        val_scaled = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
        test_scaled  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]
        categorical_features = ['hour', 'dayofweek', 'month', 'day', 'start_datetime', 'end_datetime', 'Timestamp']
        
        price_scaler = StandardScaler().fit(train_scaled[['Price']])

    return train_scaled, val_scaled, test_scaled, price_scaler

def make_lookup(df):
    lookup = defaultdict(dict)
    for _, row in df.iterrows():
        key = (row['day'], row['hour'])
        lookup[key] = row.to_dict()
    return lookup

class ElectricityDataset(Dataset):
    def __init__(self, lookup, base_dates, horizon, add_noise, first24hours):
        self.lookup = lookup
        if not first24hours:
            self.base_dates = [d + timedelta(days=1) for d in base_dates]
        else:
            self.base_dates = base_dates
        self.horizon = horizon
        self.add_noise = add_noise


    def __len__(self):
        return len(self.base_dates)

    def __getitem__(self, idx):
        base_day = self.base_dates[idx]
        np.random.seed(int(base_day.strftime("%Y%m%d")))
        feats = []
        y = []

        for hour in range(self.horizon):
            days_offset = hour // 24
            h = hour % 24

            d = (base_day + timedelta(days=days_offset)).date()
            d1 = (base_day - timedelta(days=1)).date()
            d7 = (d - timedelta(days=7))

            p_d1 = self.lookup.get((d1, h), {}).get("Price", np.nan)
            p_d7 = self.lookup.get((d7, h), {}).get("Price", np.nan)
       
            Z1 = self.lookup.get((d, h), {}).get("Load", np.nan)
            Z2 = self.lookup.get((d, h), {}).get("Solar", np.nan)
            Z3 = self.lookup.get((d, h), {}).get("WindShore", np.nan)
            Z4 = self.lookup.get((d, h), {}).get("WindOffShore", np.nan)
            Z5 = self.lookup.get((d, h), {}).get("Net_Import", np.nan)
            Z6 = self.lookup.get((d, h), {}).get("Coal_Price", np.nan)

            hour = self.lookup.get((d, h), {}).get("hour", 0)
            dayofweek = self.lookup.get((d, h), {}).get("dayofweek", 0)
            month = self.lookup.get((d, h), {}).get("month", 0)

            x = [p_d1, p_d7, Z1, Z2, Z3, Z4,Z5, Z6, hour, dayofweek, month]
            feats.append(x)

            target = self.lookup.get((d, h), {}).get("Price", np.nan)
            y.append(target)

        feats = np.array(feats, dtype=np.float32)
        y = np.array(y, dtype=np.float32)

        if self.add_noise:
            for start in range(0, self.horizon, 24):
                i = start // 24
                end = start + 24
                np.random.seed(42)
                feats[start:end, 2] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                feats[start:end, 3] *= np.random.normal(Solar_error.iloc[i, 0] + 1, Solar_error.iloc[i, 1], size=24)
                feats[start:end, 4] *= np.random.normal(WindShore_error.iloc[i, 0] + 1, WindShore_error.iloc[i, 1], size=24)
                feats[start:end, 5] *= np.random.normal(WindOffShore_error.iloc[i, 0] + 1, WindOffShore_error.iloc[i, 1], size=24)
                feats[start:end, 6] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                feats[start:end, 7] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
       
        return torch.from_numpy(feats), torch.from_numpy(y)

def build_noisy_windows(lookup, days, horizon, add_noise, first24hours):
    Xs = []
    Ys = []
    if not first24hours:
        days = [d + timedelta(days=1) for d in days]
        
    for base_day in days:
        np.random.seed(int(base_day.strftime("%Y%m%d")))
        feats = []
        y = []

        for hour in range(horizon):
            days_offset = hour // 24
            h = hour % 24

            d = (base_day + timedelta(days=days_offset)).date()
            d1 = (base_day - timedelta(days=1)).date()
            d7 = (d - timedelta(days=7))

            p_d1 = lookup.get((d1, h), {}).get("Price", np.nan)
            p_d7 = lookup.get((d7, h), {}).get("Price", np.nan)

            Z1 = lookup.get((d, h), {}).get("Load", np.nan)
            Z2 = lookup.get((d, h), {}).get("Solar", np.nan)
            Z3 = lookup.get((d, h), {}).get("WindShore", np.nan)
            Z4 = lookup.get((d, h), {}).get("WindOffShore", np.nan)
            Z5 = lookup.get((d, h), {}).get("Net_Import", np.nan)
            Z6 = lookup.get((d, h), {}).get("Coal_Price", np.nan)

            hour_val = lookup.get((d, h), {}).get("hour", 0)
            dayofweek = lookup.get((d, h), {}).get("dayofweek", 0)
            month = lookup.get((d, h), {}).get("month", 0)

            x = [p_d1, p_d7, Z1, Z2, Z3, Z4, Z5, Z6, hour_val, dayofweek, month]
            feats.append(x)

            target = lookup.get((d, h), {}).get("Price", np.nan)
            y.append(target)

        feats = np.array(feats, dtype=np.float32)
        y = np.array(y, dtype=np.float32)

        if add_noise:
            for start in range(0, horizon, 24):
                i = start // 24
                end = start + 24
                feats[start:end, 2] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                feats[start:end, 3] *= np.random.normal(Solar_error.iloc[i, 0] + 1, Solar_error.iloc[i, 1], size=24)
                feats[start:end, 4] *= np.random.normal(WindShore_error.iloc[i, 0] + 1, WindShore_error.iloc[i, 1], size=24)
                feats[start:end, 5] *= np.random.normal(WindOffShore_error.iloc[i, 0] + 1, WindOffShore_error.iloc[i, 1], size=24)
                feats[start:end, 6] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                feats[start:end, 7] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)

        Xs.append(feats.flatten()) 
        Ys.append(y)

    return np.vstack(Xs), np.vstack(Ys)
 
class LSTMForecast(nn.Module):
    def __init__(self, input_size, output_size, num_layers, dropout, hidden_size=64):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout 
        )
        self.dropout = nn.Dropout(dropout) 
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        h = h_n[-1]
        h = self.dropout(h)
        return self.fc(h)
    
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_state = None 

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_state = model.state_dict()
            return False
        
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.best_state = model.state_dict()
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        return self.early_stop

def train_and_evaluate_gbm(model_type, X_train, y_train, X_val, y_val, horizon, config):
    preds = np.zeros((len(X_val), horizon))

    for h in range(horizon):
        if model_type == "XGB":
            reg = XGBRegressor(
                n_estimators=config['n_estimators'],
                learning_rate=config['learning_rate'],
                max_depth=config['max_depth'],
                subsample=config['subsample'],
                colsample_bytree=config['colsample_bytree'],
                tree_method="hist",
                early_stopping_rounds=15,
                random_state=42,
                n_jobs=-1,
                eval_metric='rmse'
            )
            reg.fit(X_train, y_train[:, h], eval_set=[(X_val, y_val[:, h])], verbose=False)
        elif model_type == "LGBM":
            reg = LGBMRegressor(
                objective='regression',
                n_estimators=config['n_estimators'],
                learning_rate=config['learning_rate'],
                max_depth=config['max_depth'],
                subsample=config['subsample'],
                colsample_bytree=config['colsample_bytree'],
                early_stopping_rounds=15,
                random_state=42,
                n_jobs=-1,
                verbose=-1,
                eval_metric='rmse'
            )
            reg.fit(X_train, y_train[:, h].copy(), eval_set=[(X_val, y_val[:, h].copy())])
        
        preds[:, h] = reg.predict(X_val)

    val_rmse = np.sqrt(mean_squared_error(
        y_val,
        preds
    ))
    
    return val_rmse

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def train_and_tune(model_type, feature_cols,  horizon, isfirst24, **kwargs):
    if model_type == "LSTM":
        train_loader = kwargs[f'train_loader']
        val_loader   = kwargs[f'val_loader']

        param_grid = {
        'lr': [1e-3, 5e-4, 1e-4],
        'weight_decay': [1e-5, 1e-4, 1e-3],
        'dropout': [0.1, 0.2, 0.4],
        'num_layers': [1, 2, 3]
        }

        grid = list(product(*param_grid.values()))
        param_names = list(param_grid.keys())
     
        if isfirst24:
            print("Tuning for the first 24 hours")
        else:
            print(f"Tuning the remaining {horizon} hours")
        best_rmse = float('inf')
        best_config = None
        best_model_state = None
        i = 0
        for values in grid:
            i += 1
            rmses = []
            print(f"Training model {i} of {len(grid)}")
            for seed in tqdm([0, 1, 2], desc="Seeds"):
                set_seed(seed)
                params = dict(zip(param_names, values))
                model = LSTMForecast(
                        input_size=len(feature_cols),
                        output_size=horizon,
                        num_layers=params['num_layers'],
                        dropout = params['dropout'] if params['num_layers'] > 1 else 0.0
                    )
                loss_fn = nn.MSELoss()
                opt = torch.optim.Adam(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=3, min_lr=1e-5)
                early_stopper = EarlyStopping(patience=3, min_delta=1e-4)

                for epoch in range(1, 30):
                        model.train()
                        for X_batch, y_batch in train_loader:
                            opt.zero_grad()
                            y_pred = model(X_batch)
                            loss = loss_fn(y_pred, y_batch)
                            loss.backward()
                            opt.step()
                        model.eval()
                        val_loss = 0.0
                        with torch.no_grad():
                            for X_batch, y_batch in val_loader:
                                y_pred = model(X_batch)
                                val_loss += loss_fn(y_pred, y_batch).item() * X_batch.size(0)
                        val_loss /= len(val_loader.dataset)
                        scheduler.step(val_loss)
                        if early_stopper(val_loss, model):
                            break
                model.load_state_dict(early_stopper.best_state)

                model.eval()
                preds, true = [], []
                with torch.no_grad():
                    for X_batch, y in val_loader:
                        preds.append(model(X_batch))
                        true.append(y)
                preds = torch.cat(preds).cpu().numpy()
                true = torch.cat(true).cpu().numpy()
                rmse = np.sqrt(mean_squared_error(price_scaler.inverse_transform(preds),
                                                    price_scaler.inverse_transform(true)))
                rmses.append(rmse)
            avg_rmse = np.mean(rmses)
            print(f"Config {params} — Average RMSE: {avg_rmse:.4f}")    

            if avg_rmse < best_rmse:
                best_rmse = avg_rmse
                best_config = params
                best_model_state = model.state_dict()
                print(f"New best configuration: {best_config} with RMSE: {best_rmse}")

        print(f"Finished tuning with best RMSE: {best_rmse}, best_conf: {best_config}")
        print("\n")
        return best_config, best_model_state           
    else:
        X_train = kwargs['X_train']
        y_train = kwargs['y_train']
        X_val   = kwargs['X_val']
        y_val   = kwargs['y_val']
        X_test  = kwargs['X_test']

        if isfirst24:
            print("Tuning for the first 24 hours")
        else:
            print(f"Tuning the remaining {horizon} hours")

        param_grid = {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 4, 6],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
        param_names = list(param_grid.keys())
        grid = list(product(*param_grid.values()))
        best_rmse = float("inf")
        best_config = None
        i = 0
        for values in grid:
            i += 1
            print(f"Testing model {i} of {len(grid)}")
            config = dict(zip(param_names, values))
            rmses = []
            for seed in tqdm([0, 1, 2], desc="Seeds"):
                set_seed(seed)
                rmse = train_and_evaluate_gbm(model_type, X_train, y_train, X_val, y_val, horizon, config)
                rmses.append(rmse)
            
            avg_rmse = np.mean(rmses)
            print(f"Config {config} — RMSE: {avg_rmse:.4f}")
            if avg_rmse < best_rmse:
                best_rmse = avg_rmse
                best_config = config
                print(f"New best configuration: {best_config} with RMSE: {best_rmse}")

        print(f"\n Finished tuning with best config: {best_config} — RMSE: {best_rmse:.4f} \n")
        return best_config


train_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/train_dates.csv', parse_dates=['date']).sort_values(by='date')
val_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/val_dates.csv', parse_dates=['date']).sort_values(by='date')
test_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/test_dates.csv', parse_dates=['date']).sort_values(by='date')
train_dates = train_dates['date'].tolist()
val_dates = val_dates['date'].tolist()
test_dates = test_dates['date'].tolist()

feature_cols = ['price_d1','price_d7', 'Load', 'Solar', 'WindShore', 'WindOffShore', 'Net_Import', 'Coal_Price', 'hour', 'dayofweek', 'month']

train_scaled, val_scaled, test_scaled, price_scaler = load_data(FILE_PATH)
lookup_train = make_lookup(train_scaled)
lookup_val = make_lookup(val_scaled)
lookup_test = make_lookup(test_scaled)

if MODEL_TYPE == "LSTM":
    ds_train_H1 = ElectricityDataset(lookup_train, train_dates, horizon=H1, add_noise=True, first24hours = True)
    ds_val_H1   = ElectricityDataset( lookup_val, val_dates, horizon=H1, add_noise=True, first24hours = True)
    ds_test_H1  = ElectricityDataset( lookup_test, test_dates,  horizon=H1, add_noise=True, first24hours = True)
    ds_train_H2 = ElectricityDataset( lookup_train, train_dates, horizon=H2, add_noise=True, first24hours = False)
    ds_val_H2   = ElectricityDataset( lookup_val,  val_dates,   horizon=H2, add_noise=True, first24hours = False)
    ds_test_H2  = ElectricityDataset( lookup_test, test_dates,  horizon=H2, add_noise=True, first24hours = False)
    ds_train_H3 = ElectricityDataset( lookup_train, train_dates, horizon=H3, add_noise=True, first24hours = False)
    ds_val_H3   = ElectricityDataset( lookup_val,  val_dates,   horizon=H3, add_noise=True, first24hours = False)
    ds_test_H3  = ElectricityDataset( lookup_test, test_dates,  horizon=H3, add_noise=True, first24hours = False)

    train_loader_24 = DataLoader(ds_train_H1, batch_size=16, shuffle=True)
    val_loader_24   = DataLoader(ds_val_H1,   batch_size=16, shuffle=False)
    test_loader_24  = DataLoader(ds_test_H1,  batch_size=16, shuffle=False)

    start_time = time.time()
    best_config_24, best_model_state_24 = train_and_tune("LSTM", feature_cols= feature_cols, horizon =H1, isfirst24=True,
                               train_loader=train_loader_24,
                               val_loader=val_loader_24,
                               test_loader=test_loader_24)
    
    end_time_24 = time.time()
    time_24 = end_time_24 - start_time

    train_loader_H2 = DataLoader(ds_train_H2, batch_size=16, shuffle=True)
    val_loader_H2   = DataLoader(ds_val_H2,   batch_size=16, shuffle=False)
    test_loader_H2  = DataLoader(ds_test_H2,  batch_size=16, shuffle=False)

    best_config_H2, best_model_state_H2= train_and_tune("LSTM", feature_cols=feature_cols, horizon=H2, isfirst24=False,
        train_loader= train_loader_H2,
        val_loader  = val_loader_H2,
        test_loader  = test_loader_H2)
    
    end_time_H2 = time.time()
    time_H2 = end_time_H2 - end_time_24
    
    train_loader_H3 = DataLoader(ds_train_H3, batch_size=16, shuffle=True)
    val_loader_H3   = DataLoader(ds_val_H3,   batch_size=16, shuffle=False)
    test_loader_H3  = DataLoader(ds_test_H3,  batch_size=16, shuffle=False)

    best_config_H3, best_model_state_H3= train_and_tune("LSTM", feature_cols=feature_cols, horizon=H3, isfirst24=False,
        train_loader= train_loader_H3,
        val_loader  = val_loader_H3,
        test_loader  = test_loader_H3)
    
    end_time_H3 = time.time()
    time_H3 = end_time_H3 - end_time_H2
    
    configs = [
    {"name": "H1_first24", "runtime_seconds": round(time_24, 2), **best_config_24},
    {"name": "H2",         "runtime_seconds": round(time_H2, 2), **best_config_H2},
    {"name": "H3",         "runtime_seconds": round(time_H3, 2), **best_config_H3},
     ]

    df = pd.DataFrame(configs)
    df.to_csv("C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/LSTM_tuning_results.csv", index=False) 

else:
    X_train_H1, y_train_H1 = build_noisy_windows(lookup_train, train_dates, horizon=H1, add_noise=True, first24hours = True)
    X_val_H1, y_val_H1 = build_noisy_windows(lookup_val, val_dates, horizon=H1, add_noise=True, first24hours = True)
    X_test_H1, y_test_H1 = build_noisy_windows(lookup_test, test_dates, horizon=H1, add_noise=True, first24hours = True)

    X_train_H2, y_train_H2 = build_noisy_windows(lookup_train, train_dates, horizon=H2, add_noise=True, first24hours = False)
    X_val_H2, y_val_H2 = build_noisy_windows(lookup_val, val_dates, horizon=H2, add_noise=True, first24hours = False)
    X_test_H2, y_test_H2 = build_noisy_windows(lookup_test, test_dates, horizon=H2, add_noise=True, first24hours = False)

    X_train_H3, y_train_H3 = build_noisy_windows(lookup_train, train_dates, horizon=H3, add_noise=True, first24hours = False)
    X_val_H3, y_val_H3 = build_noisy_windows(lookup_val, val_dates, horizon=H3, add_noise=True, first24hours = False)
    X_test_H3, y_test_H3 = build_noisy_windows(lookup_test, test_dates, horizon=H3, add_noise=True, first24hours = False)
    
    start_time = time.time()
    best_config_24 = train_and_tune(
        MODEL_TYPE, feature_cols= feature_cols, horizon=H1, isfirst24=True,
        X_train=X_train_H1, y_train=y_train_H1,
        X_val=X_val_H1,     y_val=y_val_H1,
        X_test=X_test_H1
    )
    end_time_24 = time.time()
    time_24 = end_time_24 - start_time

    best_config_H2 = train_and_tune(
        MODEL_TYPE, feature_cols=feature_cols, horizon= H2, isfirst24=False,
        X_train=X_train_H2, y_train=y_train_H2,
        X_val=X_val_H2,     y_val=y_val_H2,
        X_test=X_test_H2
    )
    end_time_H2 = time.time()
    time_H2 = end_time_H2 - end_time_24

    best_config_H3 = train_and_tune(
        MODEL_TYPE, feature_cols=feature_cols, horizon= H3, isfirst24=False,
        X_train=X_train_H3, y_train=y_train_H3,
        X_val=X_val_H3,     y_val=y_val_H3,
        X_test=X_test_H3
    )
    end_time_H3 = time.time()
    time_H3 = end_time_H3 - end_time_H2

    configs = [
    {"name": "H1_first24", "runtime_seconds": round(time_24, 2), **best_config_24},
    {"name": "H2",         "runtime_seconds": round(time_H2, 2), **best_config_H2},
    {"name": "H3",         "runtime_seconds": round(time_H3, 2), **best_config_H3},
  ]

    df = pd.DataFrame(configs)
    df.to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/{MODEL_TYPE}_tuning_results.csv", index=False) 


LSTM, XGBoost & LGBM

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from datetime import timedelta

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from collections import defaultdict
import random

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# === CONFIGURATION ===
MODEL_TYPE       = "LGBM"   # "XGB" or "LGBM" or "LSTM"
for Forecast_Horizon in [48, 144]:     
    H1 = 24                    
    H2 = Forecast_Horizon - H1  
    FILE_PATH = 'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv'
    OUTPUT_PATH = f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/{MODEL_TYPE}_forecasts_{Forecast_Horizon}.csv'
    TRUTH_PATH = f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Models_true/{MODEL_TYPE}_true_vals_{Forecast_Horizon}.csv'
    #TRUTH_PATH = f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_validation/Truth_validation_{Forecast_Horizon}.csv'
    Validation_Path = f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_validation/{MODEL_TYPE}_forecast_validation_{Forecast_Horizon}.csv'
    load_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/load_error.csv')
    Solar_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/Solar_error.csv')
    WindShore_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindShore_error.csv')
    WindOffShore_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindOffShore_error.csv')
    all_preds = [] 
    for seed in [0, 1, 2]:
        set_seed(seed)
        def load_data(file_path):
            df = pd.read_csv(file_path, parse_dates=['start_datetime', 'end_datetime'])
        
            df['hour'] = df['start_datetime'].dt.hour
            df['dayofweek'] = df['start_datetime'].dt.dayofweek
            df['month'] = df['start_datetime'].dt.month
            df['day']= df['start_datetime'].dt.date

            if MODEL_TYPE == 'LSTM':
                train_df = df[(df['start_datetime'].dt.year == 2023)]
                val_df = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
                test_df  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]
                categorical_features = ['hour', 'dayofweek', 'month', 'day', 'start_datetime', 'end_datetime', 'Timestamp']
                features_to_scale = [col for col in train_df.columns if col not in categorical_features]

                scaler = StandardScaler()
                train_scaled = pd.DataFrame(scaler.fit_transform(train_df[features_to_scale]), index=train_df.index, columns=features_to_scale)
                val_scaled   = pd.DataFrame(scaler.transform(val_df[features_to_scale]), index=val_df.index, columns=features_to_scale)
                test_scaled  = pd.DataFrame(scaler.transform(test_df[features_to_scale]), index=test_df.index, columns=features_to_scale)

                price_scaler = StandardScaler().fit(train_df[['Price']])
                for df_scaled, df_orig in zip([train_scaled, val_scaled, test_scaled], [train_df, val_df, test_df]):
                    df_scaled['Price'] = price_scaler.transform(df_orig[['Price']])
                    df_scaled[categorical_features] = df_orig[categorical_features]

            else:
                train_scaled = df[(df['start_datetime'].dt.year == 2023)]
                val_scaled = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
                test_scaled  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]
                categorical_features = ['hour', 'dayofweek', 'month', 'day', 'start_datetime', 'end_datetime', 'Timestamp']
                
                price_scaler = StandardScaler().fit(train_scaled[['Price']])
            
            return train_scaled, val_scaled, test_scaled, price_scaler

        def make_lookup(df):
            lookup = defaultdict(dict)
            for _, row in df.iterrows():
                key = (row['day'], row['hour'])
                lookup[key] = row.to_dict()
            return lookup

        class ElectricityDataset(Dataset):
            def __init__(self, lookup, base_dates, horizon, add_noise, first24hours):
                self.lookup = lookup
                if not first24hours:
                    self.base_dates = [d + timedelta(days=1) for d in base_dates]
                else:
                    self.base_dates = base_dates
                self.horizon = horizon
                self.add_noise = add_noise


            def __len__(self):
                return len(self.base_dates)

            def __getitem__(self, idx):
                base_day = self.base_dates[idx]
                np.random.seed(int(base_day.strftime("%Y%m%d")))
                feats = []
                y = []

                for hour in range(self.horizon):
                    days_offset = hour // 24
                    h = hour % 24

                    d = (base_day + timedelta(days=days_offset)).date()
                    d1 = (base_day - timedelta(days=1)).date()
                    d7 = (d - timedelta(days=7))

                    p_d1 = self.lookup.get((d1, h), {}).get("Price", np.nan)
                    p_d7 = self.lookup.get((d7, h), {}).get("Price", np.nan)
            
                    Z1 = self.lookup.get((d, h), {}).get("Load", np.nan)
                    Z2 = self.lookup.get((d, h), {}).get("Solar", np.nan)
                    Z3 = self.lookup.get((d, h), {}).get("WindShore", np.nan)
                    Z4 = self.lookup.get((d, h), {}).get("WindOffShore", np.nan)
                    Z5 = self.lookup.get((d, h), {}).get("Net_Import", np.nan)
                    Z6 = self.lookup.get((d, h), {}).get("Coal_Price", np.nan)
            
                    hour = self.lookup.get((d, h), {}).get("hour", 0)
                    dayofweek = self.lookup.get((d, h), {}).get("dayofweek", 0)
                    month = self.lookup.get((d, h), {}).get("month", 0)

                    x = [p_d1, p_d7, Z1, Z2, Z3, Z4,Z5, Z6, hour, dayofweek, month]
                    feats.append(x)

                    target = self.lookup.get((d, h), {}).get("Price", np.nan)
                    y.append(target)

                feats = np.array(feats, dtype=np.float32)
                y = np.array(y, dtype=np.float32)

                if self.add_noise:
                    for start in range(0, self.horizon, 24):
                        i = start // 24
                        end = start + 24
                        np.random.seed(42)
                        feats[start:end, 2] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                        feats[start:end, 3] *= np.random.normal(Solar_error.iloc[i, 0] + 1, Solar_error.iloc[i, 1], size=24)
                        feats[start:end, 4] *= np.random.normal(WindShore_error.iloc[i, 0] + 1, WindShore_error.iloc[i, 1], size=24)
                        feats[start:end, 5] *= np.random.normal(WindOffShore_error.iloc[i, 0] + 1, WindOffShore_error.iloc[i, 1], size=24)
                        feats[start:end, 6] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                        feats[start:end, 7] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                    
                return torch.from_numpy(feats), torch.from_numpy(y)

        def build_noisy_windows(lookup, days, horizon, add_noise, first24hours):
            Xs = []
            Ys = []
            if not first24hours:
                days = [d + timedelta(days=1) for d in days]

            for base_day in days:
                np.random.seed(int(base_day.strftime("%Y%m%d")))
                feats = []
                y = []

                for hour in range(horizon):
                    days_offset = hour // 24
                    h = hour % 24

                    d = (base_day + timedelta(days=days_offset)).date()
                    d1 = (base_day - timedelta(days=1)).date()
                    d7 = (d - timedelta(days=7))

                    p_d1 = lookup.get((d1, h), {}).get("Price", np.nan)
                    p_d7 = lookup.get((d7, h), {}).get("Price", np.nan)

                    Z1 = lookup.get((d, h), {}).get("Load", np.nan)
                    Z2 = lookup.get((d, h), {}).get("Solar", np.nan)
                    Z3 = lookup.get((d, h), {}).get("WindShore", np.nan)
                    Z4 = lookup.get((d, h), {}).get("WindOffShore", np.nan)
                    Z5 = lookup.get((d, h), {}).get("Net_Import", np.nan)
                    Z6 = lookup.get((d, h), {}).get("Coal_Price", np.nan)

                    hour_val = lookup.get((d, h), {}).get("hour", 0)
                    dayofweek = lookup.get((d, h), {}).get("dayofweek", 0)
                    month = lookup.get((d, h), {}).get("month", 0)

                    x = [p_d1, p_d7, Z1, Z2, Z3, Z4, Z5, Z6, hour_val, dayofweek, month]
                    feats.append(x)

                    target = lookup.get((d, h), {}).get("Price", np.nan)
                    y.append(target)

                feats = np.array(feats, dtype=np.float32)
                y = np.array(y, dtype=np.float32)

                if add_noise:
                    for start in range(0, horizon, 24):
                        i = start // 24
                        end = start + 24
                        feats[start:end, 2] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                        feats[start:end, 3] *= np.random.normal(Solar_error.iloc[i, 0] + 1, Solar_error.iloc[i, 1], size=24)
                        feats[start:end, 4] *= np.random.normal(WindShore_error.iloc[i, 0] + 1, WindShore_error.iloc[i, 1], size=24)
                        feats[start:end, 5] *= np.random.normal(WindOffShore_error.iloc[i, 0] + 1, WindOffShore_error.iloc[i, 1], size=24)
                        feats[start:end, 6] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                        feats[start:end, 7] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)

                Xs.append(feats.flatten())  
                Ys.append(y)

            return np.vstack(Xs), np.vstack(Ys)
        
        class LSTMForecast(nn.Module):
            def __init__(self, input_size, output_size, num_layers, dropout, hidden_size=64):
                super().__init__()
                self.lstm = nn.LSTM(
                    input_size=input_size,
                    hidden_size=hidden_size,
                    num_layers=num_layers,
                    batch_first=True,
                    dropout=dropout  
                )
                self.dropout = nn.Dropout(dropout)  
                self.fc = nn.Linear(hidden_size, output_size)

            def forward(self, x):
                _, (h_n, _) = self.lstm(x)
                h = h_n[-1] 
                h = self.dropout(h)
                return self.fc(h)
            
        class EarlyStopping:
            def __init__(self, patience=5, min_delta=0):
                self.patience = patience
                self.min_delta = min_delta
                self.counter = 0
                self.best_loss = None
                self.early_stop = False
                self.best_state = None  

            def __call__(self, val_loss, model):
                if self.best_loss is None:
                    self.best_loss = val_loss
                    self.best_state = model.state_dict()
                    return False
                
                if val_loss < self.best_loss - self.min_delta:
                    self.best_loss = val_loss
                    self.counter = 0
                    self.best_state = model.state_dict()
                else:
                    self.counter += 1
                    print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
                    if self.counter >= self.patience:
                        self.early_stop = True
                return self.early_stop


        def train_and_predict(model_type, feature_cols,  horizon, isfirst24, **kwargs):
            if model_type == "LSTM":
                if isfirst24:
                    model = LSTMForecast(input_size=len(feature_cols), output_size=horizon, num_layers=2, dropout=0.1)
                    loss_fn = nn.MSELoss()
                    opt = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
                    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=3, min_lr=1e-5)
                    early_stopper = EarlyStopping(patience=5, min_delta=1e-4)
                else:
                    if Forecast_Horizon == 48:
                        model = LSTMForecast(input_size=len(feature_cols), output_size=horizon, num_layers=2, dropout=0.1)
                        loss_fn = nn.MSELoss()
                        opt = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
                        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=3, min_lr=1e-5)
                        early_stopper = EarlyStopping(patience=5, min_delta=1e-4)
                    else:
                        model = LSTMForecast(input_size=len(feature_cols), output_size=horizon, num_layers=3, dropout=0.1)
                        loss_fn = nn.MSELoss()
                        opt = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)
                        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=3, min_lr=1e-5)
                        early_stopper = EarlyStopping(patience=5, min_delta=1e-4)

                train_loader = kwargs[f'train_loader']
                val_loader   = kwargs[f'val_loader']
                test_loader  = kwargs[f'test_loader']

                for epoch in range(1, 30):
                    model.train()
                    train_loss = 0.0
                    for X_batch, y_batch in train_loader:
                        opt.zero_grad()
                        y_pred = model(X_batch)
                        loss = loss_fn(y_pred, y_batch)
                        loss.backward()
                        opt.step()
                        train_loss += loss.item() * X_batch.size(0)
                    train_loss /= len(train_loader.dataset)

                    model.eval()
                    val_loss = 0.0
                    with torch.no_grad():
                        for X_batch, y_batch in val_loader:
                            y_pred = model(X_batch)
                            val_loss += loss_fn(y_pred, y_batch).item() * X_batch.size(0)
                    val_loss /= len(val_loader.dataset)
                    scheduler.step(val_loss)
                    print(f"Epoch {epoch:02d} — Train: {train_loss:.4f}, Val: {val_loss:.4f}")
                    if early_stopper(val_loss, model):
                        print(f"Early stopping at epoch {epoch}")
                        break
                model.load_state_dict(early_stopper.best_state)

                model.eval()
                all_preds = []
                with torch.no_grad():
                    for X_batch, _ in test_loader:
                        all_preds.append(model(X_batch))
                preds = torch.cat(all_preds, dim=0).cpu().numpy()
                return price_scaler.inverse_transform(preds)

            else:
                X_train = kwargs['X_train']
                y_train = kwargs['y_train']
                X_val   = kwargs['X_val']
                y_val   = kwargs['y_val']
                X_test  = kwargs['X_test']

                preds = np.zeros((len(X_test), horizon))
                for h in tqdm(range(horizon), desc="Training sub-horizon"):
                    if model_type == "XGB":
                        if isfirst24:
                            reg = XGBRegressor(
                                n_estimators=100, learning_rate=0.1, max_depth=3,
                                subsample=0.8, colsample_bytree=0.8, random_state = seed,
                                tree_method="hist", eval_metric='rmse',
                                early_stopping_rounds=15, n_jobs=-1,
                            )
                        else:
                            if Forecast_Horizon == 48:
                                reg = XGBRegressor(
                                    n_estimators=200, learning_rate=0.05, max_depth=3,
                                    subsample=0.8, colsample_bytree=0.8, random_state = seed,
                                    tree_method="hist", eval_metric='rmse',
                                    early_stopping_rounds=15, n_jobs=-1
                            )
                            else:
                                reg = XGBRegressor(
                                    n_estimators=200, learning_rate=0.05, max_depth=3,
                                    subsample=0.8, colsample_bytree=0.8, random_state = seed,
                                    tree_method="hist", eval_metric='rmse',
                                    early_stopping_rounds=15, n_jobs=-1)
                        reg.fit(X_train, y_train[:, h], eval_set=[(X_val, y_val[:, h])], verbose=False)
                    elif model_type == "LGBM":
                        if isfirst24:
                            reg = LGBMRegressor(
                                objective='regression', n_estimators=100, learning_rate=0.1,
                                max_depth=3, subsample=0.8, colsample_bytree=0.8,
                                random_state=seed, eval_metric='rmse',
                                early_stopping_rounds=15, verbose=-1, n_jobs=-1
                            )
                        else:
                            if Forecast_Horizon == 48:
                                reg = LGBMRegressor(
                                    objective='regression', n_estimators=100, learning_rate=0.1,
                                    max_depth=3, subsample=0.8, colsample_bytree=0.8,
                                    random_state=seed, eval_metric='rmse',
                                    early_stopping_rounds=15, verbose=-1, n_jobs=-1
                                )
                            else: 
                                reg = LGBMRegressor(
                                    objective='regression', n_estimators=200, learning_rate=0.1,
                                    max_depth=3, subsample=0.8, colsample_bytree=0.8,
                                    random_state=seed, eval_metric='rmse',
                                    early_stopping_rounds=15, verbose=-1, n_jobs=-1
                                )
                        reg.fit(X_train, y_train[:, h].copy(), eval_set=[(X_val, y_val[:, h].copy())])
                    preds[:, h] = reg.predict(X_test)
                return preds

        train_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/train_dates.csv', parse_dates=['date']).sort_values(by='date')
        val_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/val_dates.csv', parse_dates=['date']).sort_values(by='date')
        test_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/test_dates.csv', parse_dates=['date']).sort_values(by='date')
        train_dates = train_dates['date'].tolist()
        val_dates = val_dates['date'].tolist()
        test_dates = test_dates['date'].tolist()

        feature_cols = ['price_d1','price_d7', 'Load', 'Solar', 'WindShore', 'WindOffShore', 'Net_Import', 'Coal_Price', 'hour', 'dayofweek', 'month']

        train_scaled, val_scaled, test_scaled, price_scaler = load_data(FILE_PATH)
        lookup_train = make_lookup(train_scaled)
        lookup_val = make_lookup(val_scaled)
        lookup_test = make_lookup(test_scaled)

        if MODEL_TYPE == "LSTM":
            ds_train_H1 = ElectricityDataset(lookup_train, train_dates, horizon=H1, add_noise=True, first24hours=True)
            ds_val_H1   = ElectricityDataset( lookup_val, val_dates, horizon=H1, add_noise=True, first24hours=True)
            ds_test_H1  = ElectricityDataset( lookup_test, test_dates,  horizon=H1, add_noise=True, first24hours=True)
            ds_train_H2 = ElectricityDataset( lookup_train, train_dates, horizon=H2, add_noise=True, first24hours=False)
            ds_val_H2   = ElectricityDataset(lookup_val,  val_dates,   horizon=H2, add_noise=True, first24hours=False)
            ds_test_H2  = ElectricityDataset( lookup_test, test_dates,  horizon=H2, add_noise=True, first24hours=False)

            train_loader_24 = DataLoader(ds_train_H1, batch_size=16, shuffle=True)
            val_loader_24   = DataLoader(ds_val_H1,   batch_size=16, shuffle=False)
            test_loader_24  = DataLoader(ds_test_H1,  batch_size=16, shuffle=False)

            print("Training for the first 24 hours")
            preds1 = train_and_predict("LSTM", feature_cols= feature_cols, horizon =H1, isfirst24=True,
                                    train_loader=train_loader_24,
                                    val_loader=val_loader_24,
                                    test_loader=val_loader_24)


            train_loader_H2 = DataLoader(ds_train_H2, batch_size=16, shuffle=True)
            val_loader_H2   = DataLoader(ds_val_H2,   batch_size=16, shuffle=False)
            test_loader_H2  = DataLoader(ds_test_H2,  batch_size=16, shuffle=False)

            print("\n")
            print(f"Training the other {H2} horizon")
            preds2 = train_and_predict("LSTM", feature_cols=feature_cols, horizon=H2, isfirst24=False,
                train_loader= train_loader_H2,
                val_loader  = val_loader_H2,
                test_loader  = val_loader_H2)
            y_true_H1 = np.stack([ y.numpy() for _, y in ds_val_H1 ], axis=0)  
            y_true_H2 = np.stack([ y.numpy() for _, y in ds_val_H2 ], axis=0)  

            y_true_1 = price_scaler.inverse_transform(y_true_H1)
            y_true_2 = price_scaler.inverse_transform(y_true_H2)

            y_true = np.hstack([y_true_1, y_true_2])  

        else:
            X_train_H1, y_train_H1 = build_noisy_windows(lookup_train, train_dates, horizon=H1, add_noise=True, first24hours = True)
            X_val_H1, y_val_H1 = build_noisy_windows(lookup_val, val_dates, horizon=H1, add_noise=True, first24hours = True)
            X_test_H1, y_test_H1 = build_noisy_windows(lookup_test, test_dates, horizon=H1, add_noise=True, first24hours = True)

            X_train_H2, y_train_H2 = build_noisy_windows(lookup_train, train_dates, horizon=H2, add_noise=True, first24hours = False)
            X_val_H2, y_val_H2 = build_noisy_windows(lookup_val, val_dates, horizon=H2, add_noise=True, first24hours = False)
            X_test_H2, y_test_H2 = build_noisy_windows(lookup_test, test_dates, horizon=H2, add_noise=True, first24hours = False)

            print("Training the first 24 hours")
            preds1 = train_and_predict(
                MODEL_TYPE, feature_cols= feature_cols, horizon=H1, isfirst24=True,
                X_train=X_train_H1, y_train=y_train_H1,
                X_val=X_val_H1,     y_val=y_val_H1,
                X_test=X_val_H1
            )

            print(f"Training the other {H2} hours of the forecast horizon")
            preds2 = train_and_predict(
                MODEL_TYPE, feature_cols=feature_cols, horizon= H2, isfirst24=False,
                X_train=X_train_H2, y_train=y_train_H2,
                X_val=X_val_H2,     y_val=y_val_H2,
                X_test=X_val_H2
            )
            y_true = np.hstack([y_test_H1, y_test_H2])


        y_pred = np.hstack([preds1, preds2])
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        print("\n")
        print(f"{MODEL_TYPE} Test RMSE ({Forecast_Horizon}-h avg): {rmse:.2f} €/MWh")
        all_preds.append(y_pred)

    all_preds_array = np.stack(all_preds, axis=0)
    mean_preds = np.mean(all_preds_array, axis=0)
    avg_rmse = np.sqrt(mean_squared_error(y_true, mean_preds))
    print(f"\n Avg_RMSE: {avg_rmse} \n")
    final_forecasts_df = pd.DataFrame(mean_preds, columns=[f'h+{i}' for i in range(Forecast_Horizon)])
    final_true_vals_df = pd.DataFrame(y_true, columns=[f'h+{i}' for i in range(Forecast_Horizon)])

    final_forecasts_df.to_csv(OUTPUT_PATH, index=False)
    final_true_vals_df.to_csv(TRUTH_PATH, index = False)
    #final_forecasts_df.to_csv(Validation_Path, index = False)
    


LightGBM as before, but now generates Shap values and Feature importance

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from datetime import timedelta

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from collections import defaultdict
import random
import shap

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

MODEL_TYPE       = "LGBM"  
for Forecast_Horizon in [48, 144]:     
    H1 = 24                    
    H2 = Forecast_Horizon - H1  
    FILE_PATH = 'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv'
    OUTPUT_PATH = f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/{MODEL_TYPE}_forecasts_{Forecast_Horizon}.csv'
    TRUTH_PATH = f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Models_true/{MODEL_TYPE}_true_vals_{Forecast_Horizon}.csv'
    Validation_Path = f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_validation/{MODEL_TYPE}_forecast_validation_{Forecast_Horizon}.csv'
    load_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/load_error.csv')
    Solar_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/Solar_error.csv')
    WindShore_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindShore_error.csv')
    WindOffShore_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindOffShore_error.csv')
    all_preds = [] 
    all_shap_values = []
    all_importance = []
    for seed in [0, 1, 3]:
        set_seed(seed)
        def load_data(file_path):
            df = pd.read_csv(file_path, parse_dates=['start_datetime', 'end_datetime'])
        
            df['hour'] = df['start_datetime'].dt.hour
            df['dayofweek'] = df['start_datetime'].dt.dayofweek
            df['month'] = df['start_datetime'].dt.month
            df['day']= df['start_datetime'].dt.date

            if MODEL_TYPE == 'LSTM':
                
                train_df = df[(df['start_datetime'].dt.year == 2023)]
                val_df = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
                test_df  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]
                categorical_features = ['hour', 'dayofweek', 'month', 'day', 'start_datetime', 'end_datetime', 'Timestamp']
                features_to_scale = [col for col in train_df.columns if col not in categorical_features]

                scaler = StandardScaler()
                train_scaled = pd.DataFrame(scaler.fit_transform(train_df[features_to_scale]), index=train_df.index, columns=features_to_scale)
                val_scaled   = pd.DataFrame(scaler.transform(val_df[features_to_scale]), index=val_df.index, columns=features_to_scale)
                test_scaled  = pd.DataFrame(scaler.transform(test_df[features_to_scale]), index=test_df.index, columns=features_to_scale)

                price_scaler = StandardScaler().fit(train_df[['Price']])
                for df_scaled, df_orig in zip([train_scaled, val_scaled, test_scaled], [train_df, val_df, test_df]):
                    df_scaled['Price'] = price_scaler.transform(df_orig[['Price']])
                    df_scaled[categorical_features] = df_orig[categorical_features]

            else:
                train_scaled = df[(df['start_datetime'].dt.year == 2023)]
                val_scaled = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
                test_scaled  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]
                categorical_features = ['hour', 'dayofweek', 'month', 'day', 'start_datetime', 'end_datetime', 'Timestamp']
                
                price_scaler = StandardScaler().fit(train_scaled[['Price']])
            
            return train_scaled, val_scaled, test_scaled, price_scaler

        def make_lookup(df):
            lookup = defaultdict(dict)
            for _, row in df.iterrows():
                key = (row['day'], row['hour'])
                lookup[key] = row.to_dict()
            return lookup


        def build_noisy_windows(lookup, days, horizon, add_noise, first24hours):
            Xs = []
            Ys = []
            if not first24hours:
                days = [d + timedelta(days=1) for d in days]

            for base_day in days:
                np.random.seed(int(base_day.strftime("%Y%m%d")))
                feats = []
                y = []

                for hour in range(horizon):
                    days_offset = hour // 24
                    h = hour % 24

                    d = (base_day + timedelta(days=days_offset)).date()
                    d1 = (base_day - timedelta(days=1)).date()
                    d7 = (d - timedelta(days=7))

                    p_d1 = lookup.get((d1, h), {}).get("Price", np.nan)
                    p_d7 = lookup.get((d7, h), {}).get("Price", np.nan)

                    Z1 = lookup.get((d, h), {}).get("Load", np.nan)
                    Z2 = lookup.get((d, h), {}).get("Solar", np.nan)
                    Z3 = lookup.get((d, h), {}).get("WindShore", np.nan)
                    Z4 = lookup.get((d, h), {}).get("WindOffShore", np.nan)
                    Z5 = lookup.get((d, h), {}).get("Net_Import", np.nan)
                    Z6 = lookup.get((d, h), {}).get("Coal_Price", np.nan)

                    hour_val = lookup.get((d, h), {}).get("hour", 0)
                    dayofweek = lookup.get((d, h), {}).get("dayofweek", 0)
                    month = lookup.get((d, h), {}).get("month", 0)

                    x = [p_d1, p_d7, Z1, Z2, Z3, Z4, Z5, Z6, hour_val, dayofweek, month]
                    feats.append(x)

                    target = lookup.get((d, h), {}).get("Price", np.nan)
                    y.append(target)

                feats = np.array(feats, dtype=np.float32)
                y = np.array(y, dtype=np.float32)

                if add_noise:
                    for start in range(0, horizon, 24):
                        i = start // 24
                        end = start + 24
                        feats[start:end, 2] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                        feats[start:end, 3] *= np.random.normal(Solar_error.iloc[i, 0] + 1, Solar_error.iloc[i, 1], size=24)
                        feats[start:end, 4] *= np.random.normal(WindShore_error.iloc[i, 0] + 1, WindShore_error.iloc[i, 1], size=24)
                        feats[start:end, 5] *= np.random.normal(WindOffShore_error.iloc[i, 0] + 1, WindOffShore_error.iloc[i, 1], size=24)
                        feats[start:end, 6] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                        feats[start:end, 7] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)

                Xs.append(feats.flatten()) 
                Ys.append(y)

            return np.vstack(Xs), np.vstack(Ys)
        
        def train_and_predict(model_type, feature_cols, horizon, isfirst24, **kwargs):
            X_train = kwargs['X_train']
            y_train = kwargs['y_train']
            X_val   = kwargs['X_val']
            y_val   = kwargs['y_val']
            X_test  = kwargs['X_test']

            preds = np.zeros((len(X_test), horizon))
            models = []
            importances = []
            shap_values_all = []

            for h in tqdm(range(horizon), desc="Training sub-horizon"):
                if model_type == "LGBM":
                    if isfirst24:
                        reg = LGBMRegressor(
                            objective='regression', n_estimators=100, learning_rate=0.1,
                            max_depth=3, subsample=0.8, colsample_bytree=0.8,
                            random_state=seed, eval_metric='rmse',
                            early_stopping_rounds=15, verbose=-1, n_jobs=-1
                        )
                    else:
                        if Forecast_Horizon == 48:
                            reg = LGBMRegressor(
                                objective='regression', n_estimators=100, learning_rate=0.1,
                                max_depth=3, subsample=0.8, colsample_bytree=0.8,
                                random_state=seed, eval_metric='rmse',
                                early_stopping_rounds=15, verbose=-1, n_jobs=-1
                            )
                        else:
                            reg = LGBMRegressor(
                                objective='regression', n_estimators=200, learning_rate=0.1,
                                max_depth=3, subsample=0.8, colsample_bytree=0.8,
                                random_state=seed, eval_metric='rmse',
                                early_stopping_rounds=15, verbose=-1, n_jobs=-1
                            )
                    reg.fit(X_train, y_train[:, h].copy(), eval_set=[(X_val, y_val[:, h].copy())])
                    models.append(reg)
                    importances.append(reg.feature_importances_)

                    explainer = shap.TreeExplainer(reg)
                    shap_vals = explainer.shap_values(X_test)
                    shap_values_all.append(shap_vals)

                preds[:, h] = reg.predict(X_test)

            shap_values_avg = np.mean(np.array(shap_values_all), axis=0)
            importances = np.mean(np.array(importances), axis=0)

            return preds, models, importances, shap_values_avg


        train_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/train_dates.csv', parse_dates=['date']).sort_values(by='date')
        val_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/val_dates.csv', parse_dates=['date']).sort_values(by='date')
        test_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/test_dates.csv', parse_dates=['date']).sort_values(by='date')
        train_dates = train_dates['date'].tolist()
        val_dates = val_dates['date'].tolist()
        test_dates = test_dates['date'].tolist()

        feature_cols = ['price_d1','price_d7', 'Load', 'Solar', 'WindShore', 'WindOffShore', 'Net_Import', 'Coal_Price', 'hour', 'dayofweek', 'month']

        train_scaled, val_scaled, test_scaled, price_scaler = load_data(FILE_PATH)
        lookup_train = make_lookup(train_scaled)
        lookup_val = make_lookup(val_scaled)
        lookup_test = make_lookup(test_scaled)

        X_train_H1, y_train_H1 = build_noisy_windows(lookup_train, train_dates, horizon=H1, add_noise=True, first24hours = True)
        X_val_H1, y_val_H1 = build_noisy_windows(lookup_val, val_dates, horizon=H1, add_noise=True, first24hours = True)
        X_test_H1, y_test_H1 = build_noisy_windows(lookup_test, test_dates, horizon=H1, add_noise=True, first24hours = True)

        X_train_H2, y_train_H2 = build_noisy_windows(lookup_train, train_dates, horizon=H2, add_noise=True, first24hours = False)
        X_val_H2, y_val_H2 = build_noisy_windows(lookup_val, val_dates, horizon=H2, add_noise=True, first24hours = False)
        X_test_H2, y_test_H2 = build_noisy_windows(lookup_test, test_dates, horizon=H2, add_noise=True, first24hours = False)

        print("Training the first 24 hours")
        preds1, models1, importances1, shap_vals1 = train_and_predict(
            MODEL_TYPE, feature_cols=feature_cols, horizon=H1, isfirst24=True,
            X_train=X_train_H1, y_train=y_train_H1,
            X_val=X_val_H1, y_val=y_val_H1,
            X_test=X_test_H1
        )

        print(f"Training the other {H2} hours of the forecast horizon")
        preds2, models2, importances2, shap_vals2 = train_and_predict(
            MODEL_TYPE, feature_cols=feature_cols, horizon=H2, isfirst24=False,
            X_train=X_train_H2, y_train=y_train_H2,
            X_val=X_val_H2, y_val=y_val_H2,
            X_test=X_test_H2
        )

        y_true = np.hstack([y_test_H1, y_test_H2])
        y_pred = np.hstack([preds1, preds2])

        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        print(f"{MODEL_TYPE} Test RMSE ({Forecast_Horizon}-h avg): {rmse:.2f} €/MWh")
        all_preds.append(y_pred)

        shap_vals_combined = np.hstack([shap_vals1, shap_vals2])  
        all_shap_values.append(shap_vals_combined)
        importance_combined = np.hstack([importances1, importances2]) 
        all_importance.append(importance_combined)

    all_preds_array = np.stack(all_preds, axis=0)
    mean_preds = np.mean(all_preds_array, axis=0)

    all_shap_array = np.stack(all_shap_values, axis=0)
    mean_shap_values = np.mean(all_shap_array, axis=0) 
    n_features = len(feature_cols)
    horizon =Forecast_Horizon

    all_importance_array = np.stack(all_importance, axis=0)
    mean_importance = np.mean(all_importance_array, axis=0)
    mean_importance_reshaped = mean_importance.reshape(horizon, n_features)  
    mean_importance_per_feature = mean_importance_reshaped.mean(axis=0) 
    feature_cols = np.array(feature_cols)
    mean_importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': mean_importance_per_feature
    }).sort_values(by='importance', ascending=False)

    print(mean_importance_df)
   
    mean_abs_shap = np.abs(mean_shap_values).mean(axis=0)
    mean_abs_shap_reshaped = mean_abs_shap.reshape(horizon, n_features) 
    mean_abs_shap_per_feature = mean_abs_shap_reshaped.mean(axis=0)   

    shap_summary_df = pd.DataFrame({
        'feature': feature_cols,
        'mean_abs_shap': mean_abs_shap_per_feature
    }).sort_values(by='mean_abs_shap', ascending=False)

    print("\nGlobal SHAP feature importance averaged over horizon and seeds:")
    print(shap_summary_df)

    all_preds_array = np.stack(all_preds, axis=0)
    mean_preds = np.mean(all_preds_array, axis=0)
    avg_rmse = np.sqrt(mean_squared_error(y_true, mean_preds))
    print(f"\n Avg_RMSE: {avg_rmse} \n")
    final_forecasts_df = pd.DataFrame(mean_preds, columns=[f'h+{i}' for i in range(Forecast_Horizon)])
    final_true_vals_df = pd.DataFrame(y_true, columns=[f'h+{i}' for i in range(Forecast_Horizon)])

    #final_forecasts_df.to_csv(OUTPUT_PATH, index=False)
    #final_true_vals_df.to_csv(TRUTH_PATH, index = False)


Combining LSTM, LightGBM and XGBOOSt into hybrid models (Including tuning)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import os
from sklearn.metrics import mean_squared_error
import time
from tqdm import tqdm


def compute_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def tune_values(Forecast_Horizon):
    xgb_results = pd.read_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_validation/XGB_forecast_validation_{Forecast_Horizon}.csv')
    lgbm_results = pd.read_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_validation/LGBM_forecast_validation_{Forecast_Horizon}.csv')
    lstm_results = pd.read_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_validation/LSTM_forecast_validation_{Forecast_Horizon}.csv')
    true_vals = pd.read_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_validation/Truth_validation_{Forecast_Horizon}.csv')

    xgb = xgb_results.values
    lstm = lstm_results.values
    lgbm = lgbm_results.values
    true = true_vals.values

    results = []
    for comb in ['xgb', 'lgbm']:
        print(f"Start tuning combination of LSTM and {comb} for horizon {Forecast_Horizon}")
        start_time = time.time()
        base_model = xgb if comb == 'xgb' else lgbm
        lstm_flat = lstm.flatten()

        t1_values = np.linspace(np.percentile(lstm_flat, 1), np.percentile(lstm_flat, 50), 200)
        t2_values = np.linspace(np.percentile(lstm_flat, 50), np.percentile(lstm_flat, 99), 200)

        best_rmse = float('inf')
        best_t1, best_t2 = None, None

    
        for t1 in tqdm(t1_values, desc="Tuning T1 values"):
            for t2 in t2_values:
                if t1 >= t2:
                    continue
                hybrid = np.where(lstm < t1, base_model, np.where(lstm < t2, lstm, base_model))
                rmse = compute_rmse(true, hybrid)
                if rmse < best_rmse:
                    best_rmse = rmse
                    best_t1, best_t2 = t1, t2
        
        end_time = time.time()
        run_time = end_time - start_time
        print(f"Best RMSE: {best_rmse:.4f} with T1 = {best_t1:.2f}, T2 = {best_t2:.2f}")
        results.append((comb, best_t1, best_t2, best_rmse, run_time))
    return pd.DataFrame(results, columns=["Model", "T1", "T2", "RMSE", "Run_time"])  

def tune_weighted_values(Forecast_Horizon):
    xgb_results = pd.read_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_validation/XGB_forecast_validation_{Forecast_Horizon}.csv')
    lgbm_results = pd.read_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_validation/LGBM_forecast_validation_{Forecast_Horizon}.csv')
    lstm_results = pd.read_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_validation/LSTM_forecast_validation_{Forecast_Horizon}.csv')
    true_vals = pd.read_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_validation/Truth_validation_{Forecast_Horizon}.csv')

    xgb = xgb_results.values
    lstm = lstm_results.values
    lgbm = lgbm_results.values
    true = true_vals.values

    results = []
    for comb in ['xgb', 'lgbm']:
        print(f"Start tuning combination of LSTM and {comb} for horizon {Forecast_Horizon}")
        start_time = time.time()
        base_model = xgb if comb == 'xgb' else lgbm
        W1_values = np.arange(0.0, 1.01, 0.01)
       
        best_rmse = float('inf')
        best_w1, best_w2 = None, None

    
        for w1 in tqdm(W1_values, desc="Tuning W1 values"):
            hybrid = w1 * lstm + (1 - w1) * base_model
            rmse = compute_rmse(true, hybrid)
            if rmse < best_rmse:
                best_rmse = rmse
                best_w1, best_w2 = w1, 1-w1
        
        end_time = time.time()
        run_time = end_time - start_time
        print(f"Best RMSE: {best_rmse:.4f} with T1 = {best_w1:.2f}, T2 = {best_w2:.2f}")
        results.append((comb, best_w1, best_w2, best_rmse, run_time))
    
    
    print(f"Start tuning triple combination for horizon {Forecast_Horizon}")
    start_time = time.time()

    weight_steps = np.arange(0.0, 1.01, 0.01) 

    best_rmse = float('inf')
    best_w1, best_w2, best_w3 = None, None, None

    for w1 in tqdm(weight_steps, desc="Tuning w1 values"):
        for w2 in weight_steps:
            if w1 + w2 > 1:
                continue  
            
            w3 = 1 - w1 - w2
            hybrid = w1 * lstm + w2 * xgb + w3 * lgbm
            
            rmse = compute_rmse(true, hybrid)
            
            if rmse < best_rmse:
                best_rmse = rmse
                best_w1, best_w2, best_w3 = w1, w2, w3

    end_time = time.time()
    run_time = end_time - start_time
    print(f"Best RMSE: {best_rmse:.4f} with weights w1={best_w1:.2f}, w2={best_w2:.2f}, w3={best_w3:.2f}, runtime={run_time:.2f}s")
    results.append(('Triple_combination', best_w1, best_w2, best_rmse, run_time))
    return pd.DataFrame(results, columns=["Model", "W1", "W2", "RMSE", "Run_time"])  


def generate_hybrid_forecast_threshold(Forecast_Horizon):
    lstm = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/LSTM_forecasts_{Forecast_Horizon}.csv")
    lgbm = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/LGBM_forecasts_{Forecast_Horizon}.csv")
    xgb = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/XGB_forecasts_{Forecast_Horizon}.csv")
    true_vals = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Models_true/LSTM_true_vals_{Forecast_Horizon}.csv")
    tuning_results = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_tuning_results_{Forecast_Horizon}.csv", sep=",")

    for comb in ['xgb', 'lgbm']:
        base_model = xgb if comb == 'xgb' else lgbm
        T1 = tuning_results[tuning_results['Model'] == comb]['T1'].values[0]
        T2 = tuning_results[tuning_results['Model'] == comb]['T2'].values[0]
        hybrid = np.where(lstm < T1, base_model, np.where(lstm < T2, lstm, base_model))
        rmse = compute_rmse(true_vals, hybrid)
        print(f'RMSE for Threshold combination of lstm & {comb} for forecast horizon: {Forecast_Horizon} is {rmse}')
        final_forecasts_hybrid = pd.DataFrame(hybrid, columns=[f'h+{i}' for i in range(Forecast_Horizon)])
        final_forecasts_hybrid.to_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/Hybrid_LSTM_{comb}_forecasts_{Forecast_Horizon}.csv', index=False)
        

        
def generate_hybrid_forecast_weights(Forecast_Horizon):
    lstm = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/LSTM_forecasts_{Forecast_Horizon}.csv")
    lgbm = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/LGBM_forecasts_{Forecast_Horizon}.csv")
    xgb = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/XGB_forecasts_{Forecast_Horizon}.csv")
    true_vals = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Models_true/LSTM_true_vals_{Forecast_Horizon}.csv")
    tuning_results = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_tuning_results_weighted_{Forecast_Horizon}.csv", sep=",")

    for comb in ['xgb', 'lgbm']:
        base_model = xgb if comb == 'xgb' else lgbm
        W1 = tuning_results[tuning_results['Model'] == comb]['W1'].values[0]
        W2 = tuning_results[tuning_results['Model'] == comb]['W2'].values[0]
        hybrid = W1 * lstm + W2 * base_model
        rmse = compute_rmse(true_vals, hybrid)
        print(f'RMSE for Weighted combination of lstm & {comb} for forecast horizon: {Forecast_Horizon} is {rmse}')
        final_forecasts_hybrid = pd.DataFrame(hybrid, columns=[f'h+{i}' for i in range(Forecast_Horizon)])
        final_forecasts_hybrid.to_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/Hybrid_weighted_LSTM_{comb}_forecasts_{Forecast_Horizon}.csv', index=False)

    row = tuning_results[tuning_results['Model'] == 'Triple_combination']
    W1three = row['W1'].values[0]
    W2three = row['W2'].values[0]
    W3three = 1 - W1three - W2three
    hybrid_three = W1three * lstm + W2three * xgb + W3three * lgbm
    rmse = compute_rmse(true_vals, hybrid_three)
    print(f'RMSE for Weighted combination of all three for forecast horizon: {Forecast_Horizon} is {rmse}')
    final_forecasts_hybrid_three = pd.DataFrame(hybrid_three, columns=[f'h+{i}' for i in range(Forecast_Horizon)])
    final_forecasts_hybrid_three.to_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/Hybrid_weighted_AllThree_forecasts_{Forecast_Horizon}.csv', index=False)


for Forecast_Horizon in [48, 144]:
    # Uncomment the lines below to run the tuning
    #result = tune_values(Forecast_Horizon)
    #result.to_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_tuning_results_{Forecast_Horizon}.csv', index=False)
    #result = tune_weighted_values(Forecast_Horizon)
    #result.to_csv(f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/Combination_tuning_results_weighted_{Forecast_Horizon}.csv', index=False)
    generate_hybrid_forecast_threshold(Forecast_Horizon)
    generate_hybrid_forecast_weights(Forecast_Horizon)


Tuning DDN, DDNN Johnson-SU and DDNN Mixture



In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import warnings
warnings.filterwarnings('ignore') 
import numpy as np
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR')  
import tensorflow_probability as tfp  
from tensorflow_probability import distributions as tfd
from datetime import datetime, timedelta
keras = tf.keras
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy.optimize import brentq
from tqdm import tqdm
from collections import defaultdict
from keras import regularizers
import time
import random
from keras import backend as K
import gc


distribution = True #False for DNN point forecasts, True for probabilistic forecasts
FILE_PATH = 'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv'
mixture = False # True for mixture density, False for regular Johnson-SU distribution
load_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/load_error.csv')
Solar_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/Solar_error.csv')
WindShore_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindShore_error.csv')
WindOffShore_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindOffShore_error.csv')

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

def calc_rmse(y_true, y_pred):
    rmse_val = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"RMSE: {rmse_val:.3f} €/MWh")
    return rmse_val

def pinball_loss(y_true, y_pred, quantiles):
   
    N, H, Q = y_pred.shape
    losses = np.zeros((N, H, Q))
    for q_idx, q in enumerate(quantiles):
        err = y_true - y_pred[:, :, q_idx]
        losses[:, :, q_idx] = np.maximum(q * err, (q - 1) * err)
    return losses

def calculate_pinball_loss_CRPS(y_true, Y_pred, quantiles):
    pinball_losses = pinball_loss(np.array(y_true), Y_pred, np.array(quantiles))
    crps = np.mean(np.trapz(pinball_losses, np.array(quantiles), axis=-1))
    return pinball_losses, np.mean(pinball_losses), crps

def winkler_score(y_true, y_lower, y_upper, alpha):
    width = y_upper - y_lower
    under = (y_lower - y_true) * (y_true < y_lower)
    over  = (y_true - y_upper) * (y_true > y_upper)
    
    score = width + (2.0 / alpha) * (under + over)
    return score

def calculate_winkler(Y_pred, y_true, quantiles, lower_q, upper_q):
    alpha = 1.0 - (upper_q - lower_q)
    try:
        li = quantiles.index(lower_q)
        ui = quantiles.index(upper_q)
    except ValueError:
        raise ValueError(f"lower_q={lower_q} or upper_q={upper_q} not in quantiles list")
    
    y_lower = Y_pred[:, :, li]
    y_upper = Y_pred[:, :, ui]
    y_true = np.array(y_true)
    
    scores = winkler_score(y_true, y_lower, y_upper, alpha)
    mean_score = scores.mean()
    print(f"Mean Winkler score (α={alpha:.2f}, interval {lower_q}-{upper_q}): {mean_score:.4f}")
    return scores, mean_score


def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['start_datetime', 'end_datetime'])
  
    df['hour'] = df['start_datetime'].dt.hour
    df['dayofweek'] = df['start_datetime'].dt.dayofweek
    df['month'] = df['start_datetime'].dt.month
    df['day']= df['start_datetime'].dt.date

    train_df = df[(df['start_datetime'].dt.year == 2023)]
    val_df = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
    test_df  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]

    categorical_features = ['hour', 'dayofweek', 'month', 'day', 'start_datetime', 'end_datetime', 'Timestamp']
    features_to_scale = [col for col in train_df.columns if col not in categorical_features]

    scaler = StandardScaler()
    train_scaled = pd.DataFrame(scaler.fit_transform(train_df[features_to_scale]), index=train_df.index, columns=features_to_scale)
    val_scaled   = pd.DataFrame(scaler.transform(val_df[features_to_scale]), index=val_df.index, columns=features_to_scale)
    test_scaled  = pd.DataFrame(scaler.transform(test_df[features_to_scale]), index=test_df.index, columns=features_to_scale)

    price_scaler = StandardScaler().fit(train_df[['Price']])
    for df_scaled, df_orig in zip([train_scaled, val_scaled, test_scaled], [train_df, val_df, test_df]):
        df_scaled['Price'] = price_scaler.transform(df_orig[['Price']])
        df_scaled[categorical_features] = df_orig[categorical_features]

    return train_scaled, val_scaled, test_scaled, price_scaler

def make_lookup(df):
    lookup = defaultdict(dict)
    for _, row in df.iterrows():
        key = (row['day'], row['hour'])
        lookup[key] = row.to_dict()
    return lookup

def build_noisy_windows(lookup, days, horizon, add_noise):
    Xs = []
    Ys = []

    for base_day in days:
        np.random.seed(int(base_day.strftime("%Y%m%d")))
        feats = []
        y = []

        for hour in range(horizon):
            days_offset = hour // 24
            h = hour % 24

            d = (base_day + timedelta(days=days_offset)).date()
            d1 = (base_day - timedelta(days=1)).date()
            d7 = (d - timedelta(days=7))

            p_d1 = lookup.get((d1, h), {}).get("Price", np.nan)
            p_d7 = lookup.get((d7, h), {}).get("Price", np.nan)

            Z1 = lookup.get((d, h), {}).get("Load", np.nan)
            Z2 = lookup.get((d, h), {}).get("Solar", np.nan)
            Z3 = lookup.get((d, h), {}).get("WindShore", np.nan)
            Z4 = lookup.get((d, h), {}).get("WindOffShore", np.nan)
            Z5 = lookup.get((d, h), {}).get("Net_Import", np.nan)
            Z6 = lookup.get((d, h), {}).get("Coal_Price", np.nan)

            hour_val = lookup.get((d, h), {}).get("hour", 0)
            dayofweek = lookup.get((d, h), {}).get("dayofweek", 0)
            month = lookup.get((d, h), {}).get("month", 0)

            x = [p_d1, p_d7, Z1, Z2, Z3, Z4, Z5, Z6, hour_val, dayofweek, month]
            feats.append(x)

            target = lookup.get((d, h), {}).get("Price", np.nan)
            y.append(target)

        feats = np.array(feats, dtype=np.float32)
        y = np.array(y, dtype=np.float32)

        if add_noise:
            for start in range(0, horizon, 24):
                i = start // 24
                end = start + 24
                feats[start:end, 2] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                feats[start:end, 3] *= np.random.normal(Solar_error.iloc[i, 0] + 1, Solar_error.iloc[i, 1], size=24)
                feats[start:end, 4] *= np.random.normal(WindShore_error.iloc[i, 0] + 1, WindShore_error.iloc[i, 1], size=24)
                feats[start:end, 5] *= np.random.normal(WindOffShore_error.iloc[i, 0] + 1, WindOffShore_error.iloc[i, 1], size=24)
                feats[start:end, 6] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                feats[start:end, 7] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)

        Xs.append(feats.flatten())  
        Ys.append(y)

    return np.vstack(Xs), np.vstack(Ys)



def build_model(input_dim, horizon, distribution, dropout_rates, activations, layer_sizes, learning_and_weight_rate, regularization, mixture):
    inputs = keras.Input(shape=(input_dim,), name='input_layer')
    
    hidden = keras.layers.Dense(layer_sizes[0], 
                               activation=activations[0], 
                               kernel_regularizer=regularization[0])(inputs)
    hidden = keras.layers.Dropout(dropout_rates[0])(hidden)
    hidden = keras.layers.Dense(layer_sizes[1], 
                               activation=activations[1], 
                               kernel_regularizer=regularization[1])(hidden)
    hidden = keras.layers.Dropout(dropout_rates[1])(hidden)

    if not distribution:
        outputs = keras.layers.Dense(horizon, activation='linear')(hidden)
        model = keras.Model(inputs, outputs)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_and_weight_rate[0], weight_decay=learning_and_weight_rate[1]),
                      loss='mse',
                      metrics=[keras.metrics.RootMeanSquaredError()])
        return model
    
    if distribution:
        if not mixture:
            param_layers = []
            for p in range(4):
                param_layers.append(keras.layers.Dense(
                    horizon, activation='linear')(hidden))
            linear = tf.keras.layers.concatenate(param_layers)
            outputs = tfp.layers.DistributionLambda(
                lambda t: tfd.JohnsonSU(
                        loc=t[..., :horizon],
                        scale=1e-3 + 3 * tf.math.softplus(t[..., horizon:2*horizon]),
                        tailweight= 1 + 3 * tf.math.softplus(t[..., 2*horizon:3*horizon]),
                        skewness=t[..., 3*horizon:]))(linear)
        
            model = keras.Model(inputs, outputs)

            model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_and_weight_rate[0], weight_decay=learning_and_weight_rate[1]),
                        loss=lambda y, rv_y: -rv_y.log_prob(y),
                        metrics=['mae'])
            return model
        else:
            param_output = keras.layers.Dense(9 * horizon)(hidden) 

            def make_mixture_distribution(params):
                params = tf.reshape(params, [-1, horizon, 9])

                loc1 = params[:, :, 0]
                scale1 = 1e-3 + 1.5 * tf.nn.softplus(params[:, :, 1])
                tail1 = 1 + 1.5 * tf.nn.softplus(params[:, :, 2])
                skew1 = params[:, :, 3]

                loc2 = params[:, :, 4]
                scale2 = 1e-3 + 1.5 * tf.nn.softplus(params[:, :, 5])
                tail2 = 1 + 1.5 * tf.nn.softplus(params[:, :, 6])
                skew2 = params[:, :, 7]

                raw_logits = params[:, :, 8]  
                logits = tf.stack([raw_logits, -raw_logits], axis=-1)  

                locs = tf.stack([loc1, loc2], axis=-1)
                scales = tf.stack([scale1, scale2], axis=-1)
                tails = tf.stack([tail1, tail2], axis=-1)
                skews = tf.stack([skew1, skew2], axis=-1)

                components = tfd.JohnsonSU(
                    loc=locs,
                    scale=scales,
                    tailweight=tails,
                    skewness=skews
                )

                mixture_dist = tfd.MixtureSameFamily(
                    mixture_distribution=tfd.Categorical(logits=logits),
                    components_distribution=components
                )

                return mixture_dist

            outputs = tfp.layers.DistributionLambda(make_mixture_distribution)(param_output)

            model = keras.Model(inputs, outputs)
            model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_and_weight_rate[0], weight_decay=learning_and_weight_rate[1]),
                        loss=lambda y, rv_y: -rv_y.log_prob(y),
                        metrics=['mae'])
            return model 
    
def extract_distribution_parameters(model, test_ds):
        locs = []
        scales = []
        tailweights = []
        skewnesses = []

        for x_batch, _ in test_ds:
            dist = model(x_batch, training=False)
            locs.append(dist.loc.numpy())
            scales.append(dist.scale.numpy())
            tailweights.append(dist.tailweight.numpy())
            skewnesses.append(dist.skewness.numpy())

        locs = np.concatenate(locs, axis=0)
        scales = np.concatenate(scales, axis=0)
        tailweights = np.concatenate(tailweights, axis=0)
        skewnesses = np.concatenate(skewnesses, axis=0)

        return locs, scales, tailweights, skewnesses

def extract_distribution_parameters_mixture(model, test_ds):
    locs = []
    scales = []
    tailweights = []
    skewnesses = []
    logits = []

    for x_batch, _ in test_ds:
        dist = model(x_batch, training=False)
        locs.append(dist.components_distribution.loc.numpy())
        scales.append(dist.components_distribution.scale.numpy())
        tailweights.append(dist.components_distribution.tailweight.numpy())
        skewnesses.append(dist.components_distribution.skewness.numpy())
        logits.append(dist.mixture_distribution.logits.numpy())

    locs = np.concatenate(locs, axis=0)
    scales = np.concatenate(scales, axis=0)
    tailweights = np.concatenate(tailweights, axis=0)
    skewnesses = np.concatenate(skewnesses, axis=0)
    logits = np.concatenate(logits, axis=0)
    

    return locs, scales, tailweights, skewnesses, logits


    
def hyperparameter_search(X_train, y_train, X_val, y_val, input_dim, horizon, distribution, price_scaler, mixture):
    
    dropout_options = [
    [0.0, 0.0],
    [0.1, 0.1],
    [0.25, 0.25],
    [0.1, 0.25],    
    [0.25, 0.1]
    ]

    activation_options = [
    ['relu', 'relu'],
    ['elu', 'elu'],
    ['tanh', 'tanh']]

 
    layer_size_options = [
        [256, 128], 
        [512, 256]  
    ]

    learning_rate_weight_decay_options = [
        [1e-3, 0],
        [1e-3, 1e-5],
        [5e-4, 1e-5] 
    ]

    regularization_options = [
        [regularizers.l2(1e-5), regularizers.l2(1e-5)],
        [regularizers.l2(1e-4), regularizers.l2(1e-4)]
    ]
    
    if not distribution:
        best_rmse = np.inf
        best_params = None
        best_model = None
        numb_options = len(dropout_options) * len(activation_options) * len(layer_size_options) * len(learning_rate_weight_decay_options) * len(regularization_options)

        i = 0
        for dropout_rates in dropout_options:
            for activations in activation_options:
                for layer_sizes in layer_size_options:
                    for learning_and_weight in learning_rate_weight_decay_options:
                        for regularization in regularization_options:
                            i += 1
                            rmses = []
                            reg_values = [reg.l2 if reg is not None else None for reg in regularization]
                            print(f"\nTraining model {i} of {numb_options} with dropout={dropout_rates}, activations={activations}, layer_sizes={layer_sizes}, learning&weight ={learning_and_weight} and regularization = {reg_values}")

                            for seed in tqdm([0,1,2], desc="Seeds", disable = True):
                                set_seed(seed)

                                model = build_model(input_dim, horizon, distribution,
                                                    dropout_rates=dropout_rates,
                                                    activations=activations,
                                                    layer_sizes=layer_sizes,
                                                    learning_and_weight_rate=learning_and_weight,
                                                    regularization=regularization, mixture=mixture)
                                
                                early_stopping = tf.keras.callbacks.EarlyStopping(
                                    monitor='val_loss',  
                                    patience=4,                          
                                    restore_best_weights=True            
                                )

                                model.fit(X_train, y_train,
                                                    validation_data=(X_val, y_val),
                                                    epochs=25,
                                                    batch_size=16,
                                                    callbacks=[early_stopping],
                                                    verbose=0)
                                
                                pred = model.predict(X_val)
                                y_pred = price_scaler.inverse_transform(pred)
                                y_true = price_scaler.inverse_transform(y_val).round(2)

                                val_rmse = np.sqrt(mean_squared_error(y_pred, y_true))
                                rmses.append(val_rmse)
                            avg_rmse = np.mean(rmses)    
                            print(f"Validation RMSE: {avg_rmse:.4f}")

                            if avg_rmse < best_rmse:
                                best_rmse = avg_rmse
                                best_params = (dropout_rates, activations, layer_sizes, learning_and_weight, reg_values)
                                best_model = model
                                print(f"New best model with validation RMSE: {best_rmse}")

        print(f"\nFinished Tuning! Best validation RMSE: {best_rmse:.4f}")
        print(f"Best params: dropout={best_params[0]}, activations={best_params[1]}, layer_sizes={best_params[2]}, learning&weight={best_params[3]}, regularization={best_params[4]}")

        return best_model, best_params, best_rmse
    
    if distribution:
        val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(16)
        best_crps = np.inf
        best_params = None
        best_model = None
        numb_options = len(dropout_options) * len(activation_options) * len(layer_size_options) * len(learning_rate_weight_decay_options) * len(regularization_options)

        i = 0
        for dropout_rates in dropout_options:
            for activations in activation_options:
                for layer_sizes in layer_size_options:
                    for learning_and_weight in learning_rate_weight_decay_options:
                        for regularization in regularization_options:
                            i += 1
                            crpses = []
                            reg_values = [reg.l2 if reg is not None else None for reg in regularization]
                            print(f"\nTraining model {i} of {numb_options} with dropout={dropout_rates}, activations={activations}, layer_sizes={layer_sizes}, learning&weight ={learning_and_weight} and regularization = {reg_values}")

                            for seed in tqdm([0,1,3], desc="Seeds", disable=True):
                                set_seed(seed)

                                model = build_model(input_dim, horizon, distribution,
                                                    dropout_rates=dropout_rates,
                                                    activations=activations,
                                                    layer_sizes=layer_sizes,
                                                    learning_and_weight_rate=learning_and_weight,
                                                    regularization=regularization, mixture =mixture)
                                
                                early_stopping = tf.keras.callbacks.EarlyStopping(
                                    monitor='val_loss',  
                                    patience=4,                          
                                    restore_best_weights=True            
                                )

                                model.fit(X_train, y_train,
                                                    validation_data=(X_val, y_val),
                                                    epochs=20,
                                                    batch_size=16,
                                                    callbacks=[early_stopping],
                                                    verbose=0)
                                
                                if not mixture:
                                    locs, scales, tailweights, skewnesses = extract_distribution_parameters(model, val_ds)
                                    distributions = tfd.JohnsonSU(
                                    loc=np.array(locs),
                                    scale=np.array(scales),
                                    tailweight=np.array(tailweights),
                                    skewness=np.array(skewnesses)
                                )
                                    
                                    n_samples = 2000
                                    samples = distributions.sample(n_samples) 

                                    quantiles = np.arange(0.01, 1.00, 0.01).round(2)
                                    samples_np = samples.numpy()  

                                    Y_pred = np.quantile(samples_np, q=quantiles, axis=0)  

                                    Y_pred = np.transpose(Y_pred, (1, 2, 0)) 
                                    (N, H, Q)  = Y_pred.shape

                                
                                    Y_pred = price_scaler.inverse_transform(Y_pred.reshape(-1,1)).reshape(N, H, Q)
                                    y_true = price_scaler.inverse_transform(y_val).round(2)

                                else:
                                    locs, scales, tailweights, skewnesses, logits = extract_distribution_parameters_mixture(model, val_ds)
                                    distributions = tfd.JohnsonSU(
                                    loc=np.array(locs),
                                    scale=np.array(scales),
                                    tailweight=np.array(tailweights),
                                    skewness=np.array(skewnesses),
                                )
                                    mixture_distribution = tfd.MixtureSameFamily(
                                        mixture_distribution=tfd.Categorical(logits=logits),
                                        components_distribution=distributions,
                                    )
                                    n_samples = 1000
                                    samples = mixture_distribution.sample(n_samples)  

                                    quantiles = np.arange(0.01, 1.00, 0.01).round(2)
                                    samples_np = samples.numpy()  

                                    Y_pred = np.quantile(samples_np, q=quantiles, axis=0)  

                                    Y_pred = np.transpose(Y_pred, (1, 2, 0))  
                                    (N, H, q)  = Y_pred.shape

                                    Y_pred = price_scaler.inverse_transform(Y_pred.reshape(-1,1)).reshape(N, H, len(quantiles))
                                    y_true = price_scaler.inverse_transform(y_val).round(2)
                                _, _, val_crps = calculate_pinball_loss_CRPS(y_true, Y_pred, quantiles)
                                crpses.append(val_crps)
                                print(f"CRPS of seed {seed} is {val_crps}")
                                del model
                                del distributions, locs, scales, tailweights, skewnesses, Y_pred
                                K.clear_session()
                                gc.collect()
                            avg_crps = np.mean(crpses)    
                            print(f"Validation CRPS: {avg_crps:.4f}")

                            if avg_crps < best_crps:
                                best_crps= avg_crps
                                best_params = (dropout_rates, activations, layer_sizes, learning_and_weight, reg_values)
                                print(f"New best model with validation CRPS: {best_crps}")

        print(f"\nFinished Tuning! Best validation CRPS: {best_crps:.4f}")
        print(f"Best params: dropout={best_params[0]}, activations={best_params[1]}, layer_sizes={best_params[2]}, learning&weight={best_params[3]}, regularization={best_params[4]}")

        return best_params, best_crps

def prepare_and_train(distribution, Forecast_Horizon):
    train_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/train_dates.csv', parse_dates=['date']).sort_values(by='date')
    val_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/val_dates.csv', parse_dates=['date']).sort_values(by='date')
    test_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/test_dates.csv', parse_dates=['date']).sort_values(by='date')
    train_dates = train_dates['date'].tolist()
    val_dates = val_dates['date'].tolist()
    test_dates = test_dates['date'].tolist()

    train_scaled, val_scaled, test_scaled, price_scaler = load_data(FILE_PATH)
    lookup_train = make_lookup(train_scaled)
    lookup_val = make_lookup(val_scaled)
    lookup_test = make_lookup(test_scaled)

    x_train, y_train = build_noisy_windows(lookup_train, train_dates, Forecast_Horizon, add_noise=True)
    x_val, y_val = build_noisy_windows(lookup_val, val_dates, Forecast_Horizon, add_noise=True)
    x_test, y_test= build_noisy_windows(lookup_test, test_dates, Forecast_Horizon, add_noise = True)

    model = build_model(input_dim=x_train.shape[1], horizon=Forecast_Horizon, distribution=distribution)

    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(16)
    val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(16)
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(16)

    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(train_ds, epochs=30, validation_data=val_ds, callbacks=[early_stop])

    if not distribution:
        preds = model.predict(test_ds)
        return preds, y_test, price_scaler, test_dates
    else:
        if not mixture:
            def extract_distribution_parameters(model, test_ds, horizon):
                locs = []
                scales = []
                tailweights = []
                skewnesses = []

                for x_batch, _ in test_ds:
                    dist = model(x_batch, training=False)
                    locs.append(dist.loc.numpy())
                    scales.append(dist.scale.numpy())
                    tailweights.append(dist.tailweight.numpy())
                    skewnesses.append(dist.skewness.numpy())

                locs = np.concatenate(locs, axis=0)
                scales = np.concatenate(scales, axis=0)
                tailweights = np.concatenate(tailweights, axis=0)
                skewnesses = np.concatenate(skewnesses, axis=0)

                return locs, scales, tailweights, skewnesses
            locs, scales, tailweights, skewnesses = extract_distribution_parameters(model, test_ds, horizon=Forecast_Horizon)
            return locs, scales, tailweights, skewnesses, y_test, price_scaler, test_dates

        else:
            def extract_distribution_parameters(model, test_ds, horizon):
                locs = []
                scales = []
                tailweights = []
                skewnesses = []
                logits = []

                for x_batch, _ in test_ds:
                    dist = model(x_batch, training=False)
                    locs.append(dist.components_distribution.loc.numpy())
                    scales.append(dist.components_distribution.scale.numpy())
                    tailweights.append(dist.components_distribution.tailweight.numpy())
                    skewnesses.append(dist.components_distribution.skewness.numpy())
                    logits.append(dist.mixture_distribution.logits.numpy())

                locs = np.concatenate(locs, axis=0)
                scales = np.concatenate(scales, axis=0)
                tailweights = np.concatenate(tailweights, axis=0)
                skewnesses = np.concatenate(skewnesses, axis=0)
                logits = np.concatenate(logits, axis=0)
                

                return locs, scales, tailweights, skewnesses, logits
       
            locs, scales, tailweights, skewnesses, logits = extract_distribution_parameters(model, test_ds, horizon=Forecast_Horizon)
            return locs, scales, tailweights, skewnesses, logits, y_test, price_scaler, test_dates


if not distribution:
    results = []
    for Forecast_horizon in [48, 144]:
        print(f"\nStart tuning for Forecast horizon: {Forecast_horizon}")
        start_time = time.time()
        train_scaled, val_scaled, test_scaled, price_scaler = load_data(FILE_PATH)
        lookup_train = make_lookup(train_scaled)
        lookup_val = make_lookup(val_scaled)

        train_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/train_dates.csv', parse_dates=['date']).sort_values(by='date')
        val_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/val_dates.csv', parse_dates=['date']).sort_values(by='date')
        train_dates = train_dates['date'].tolist()
        val_dates = val_dates['date'].tolist()


        X_train, y_train = build_noisy_windows(lookup_train, train_dates, Forecast_horizon, add_noise = True)
        X_val, y_val = build_noisy_windows(lookup_val, val_dates, Forecast_horizon, add_noise = True)

        input_dim = X_train.shape[1]

        best_model, best_params, best_rmse = hyperparameter_search(X_train, y_train, X_val, y_val, input_dim, Forecast_horizon, distribution, price_scaler, mixture)
        end_time = time.time()
        run_time = end_time - start_time
        results.append((Forecast_horizon, best_params, best_rmse, run_time))
    results_df = pd.DataFrame(results, columns=['Forecast Horizon', 'Best Params', 'Best RMSE', 'Run time'])
    results_df.to_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/DDNN_point_tuning_results.csv', index=False)
if distribution:
    results = []
    for Forecast_horizon in [48, 144]:
        print(f"\nStart tuning probability model for Forecast horizon: {Forecast_horizon}")
        start_time = time.time()
        train_scaled, val_scaled, test_scaled, price_scaler = load_data(FILE_PATH)
        lookup_train = make_lookup(train_scaled)
        lookup_val = make_lookup(val_scaled)

        train_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/train_dates.csv', parse_dates=['date']).sort_values(by='date')
        val_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/val_dates.csv', parse_dates=['date']).sort_values(by='date')
        train_dates = train_dates['date'].tolist()
        val_dates = val_dates['date'].tolist()


        X_train, y_train = build_noisy_windows(lookup_train, train_dates, Forecast_horizon, add_noise = True)
        X_val, y_val = build_noisy_windows(lookup_val, val_dates, Forecast_horizon, add_noise = True)

        input_dim = X_train.shape[1]

        best_params, best_crps = hyperparameter_search(X_train, y_train, X_val, y_val, input_dim, Forecast_horizon, distribution, price_scaler, mixture)
        end_time = time.time()
        run_time = end_time - start_time
        results.append((Forecast_horizon, best_params, best_crps, run_time))
    results_df = pd.DataFrame(results, columns=['Forecast Horizon', 'Best Params', 'Best CRPS', 'Run time'])
    results_df.to_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Tuning/DDNN_prob_tuning_results_144.csv', index=False)

    

DDN point, DDNN Johnson-SU and DDNN Mixture

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.compat.v2.enable_v2_behavior()
import tensorflow_probability as tfp  
from tensorflow_probability import distributions as tfd
from datetime import datetime, timedelta
keras = tf.keras
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy.optimize import brentq
from tqdm import tqdm
from collections import defaultdict
import ast
from keras import regularizers

for Forecast_Horizon in [48,144]: 
    distribution = True  # False for point forecasts, True for distributional forecasts
    FILE_PATH = 'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv'
    OUTPUT_PATH = f'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/DDNN_samples_{Forecast_Horizon}.csv'
    mixture = False # False for single Johnson-SU, True for mixture of distributions
    load_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/load_error.csv')
    Solar_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/Solar_error.csv')
    WindShore_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindShore_error.csv')
    WindOffShore_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindOffShore_error.csv')

    all_preds = [] 
    for seed in [0, 1, 2, 4]:

        def calc_rmse(y_true, y_pred):
            rmse_val = np.sqrt(mean_squared_error(y_true, y_pred))
            print(f"RMSE: {rmse_val:.3f} €/MWh")
            return rmse_val

        def pinball_loss(y_true, y_pred, quantiles):
        
            N, H, Q = y_pred.shape
            losses = np.zeros((N, H, Q))
            for q_idx, q in enumerate(quantiles):
                err = y_true - y_pred[:, :, q_idx]
                losses[:, :, q_idx] = np.maximum(q * err, (q - 1) * err)
            return losses

        def calculate_pinball_loss_CRPS(y_true, Y_pred, quantiles):
            pinball_losses = pinball_loss(np.array(y_true), Y_pred, np.array(quantiles))
            print(f"Mean Pinball Loss: {np.mean(pinball_losses):.4f}")
            crps = np.mean(np.trapz(pinball_losses, np.array(quantiles), axis=-1))
            print(f'Approx CRPS: {crps:.4f}')
            return pinball_losses, np.mean(pinball_losses), crps

        def winkler_score(y_true, y_lower, y_upper, alpha):
            width = y_upper - y_lower
            
            under = (y_lower - y_true) * (y_true < y_lower)
            over  = (y_true - y_upper) * (y_true > y_upper)
            
            score = width + (2.0 / alpha) * (under + over)
            return score

        def calculate_winkler(Y_pred, y_true, quantiles, lower_q, upper_q):
            alpha = 1.0 - (upper_q - lower_q)
            try:
                li = quantiles.index(lower_q)
                ui = quantiles.index(upper_q)
            except ValueError:
                raise ValueError(f"lower_q={lower_q} or upper_q={upper_q} not in quantiles list")
            
            y_lower = Y_pred[:, :, li]
            y_upper = Y_pred[:, :, ui]
            y_true = np.array(y_true)
            
            scores = winkler_score(y_true, y_lower, y_upper, alpha)
            mean_score = scores.mean()
            print(f"Mean Winkler score (α={alpha:.2f}, interval {lower_q}-{upper_q}): {mean_score:.4f}")
            return scores, mean_score

        def load_data(file_path):
            df = pd.read_csv(file_path, parse_dates=['start_datetime', 'end_datetime'])
        
            df['hour'] = df['start_datetime'].dt.hour
            df['dayofweek'] = df['start_datetime'].dt.dayofweek
            df['month'] = df['start_datetime'].dt.month
            df['day']= df['start_datetime'].dt.date

            train_df = df[(df['start_datetime'].dt.year == 2023)]
            val_df = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
            test_df  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]

            categorical_features = ['hour', 'dayofweek', 'month', 'day', 'start_datetime', 'end_datetime', 'Timestamp']
            features_to_scale = [col for col in train_df.columns if col not in categorical_features]

            scaler = StandardScaler()
            train_scaled = pd.DataFrame(scaler.fit_transform(train_df[features_to_scale]), index=train_df.index, columns=features_to_scale)
            val_scaled   = pd.DataFrame(scaler.transform(val_df[features_to_scale]), index=val_df.index, columns=features_to_scale)
            test_scaled  = pd.DataFrame(scaler.transform(test_df[features_to_scale]), index=test_df.index, columns=features_to_scale)

            price_scaler = StandardScaler().fit(train_df[['Price']])
            for df_scaled, df_orig in zip([train_scaled, val_scaled, test_scaled], [train_df, val_df, test_df]):
                df_scaled['Price'] = price_scaler.transform(df_orig[['Price']])
                df_scaled[categorical_features] = df_orig[categorical_features]

            return train_scaled, val_scaled, test_scaled, price_scaler

        def make_lookup(df):
            lookup = defaultdict(dict)
            for _, row in df.iterrows():
                key = (row['day'], row['hour'])
                lookup[key] = row.to_dict()
            return lookup

        def build_noisy_windows(lookup, days, horizon, add_noise):
            Xs = []
            Ys = []

            for base_day in days:
                np.random.seed(int(base_day.strftime("%Y%m%d")))
                feats = []
                y = []

                for hour in range(horizon):
                    days_offset = hour // 24
                    h = hour % 24

                    d = (base_day + timedelta(days=days_offset)).date()
                    d1 = (base_day - timedelta(days=1)).date()
                    d7 = (d - timedelta(days=7))

                    p_d1 = lookup.get((d1, h), {}).get("Price", np.nan)
                    p_d7 = lookup.get((d7, h), {}).get("Price", np.nan)

                    Z1 = lookup.get((d, h), {}).get("Load", np.nan)
                    Z2 = lookup.get((d, h), {}).get("Solar", np.nan)
                    Z3 = lookup.get((d, h), {}).get("WindShore", np.nan)
                    Z4 = lookup.get((d, h), {}).get("WindOffShore", np.nan)
                    Z5 = lookup.get((d, h), {}).get("Net_Import", np.nan)
                    Z6 = lookup.get((d, h), {}).get("Coal_Price", np.nan)


                    hour_val = lookup.get((d, h), {}).get("hour", 0)
                    dayofweek = lookup.get((d, h), {}).get("dayofweek", 0)
                    month = lookup.get((d, h), {}).get("month", 0)

                    x = [p_d1, p_d7, Z1, Z2, Z3, Z4, Z5, Z6, hour_val, dayofweek, month]
                    feats.append(x)

                    target = lookup.get((d, h), {}).get("Price", np.nan)
                    y.append(target)

                feats = np.array(feats, dtype=np.float32)
                y = np.array(y, dtype=np.float32)

                if add_noise:
                    for start in range(0, horizon, 24):
                        i = start // 24
                        end = start + 24
                        feats[start:end, 2] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                        feats[start:end, 3] *= np.random.normal(Solar_error.iloc[i, 0] + 1, Solar_error.iloc[i, 1], size=24)
                        feats[start:end, 4] *= np.random.normal(WindShore_error.iloc[i, 0] + 1, WindShore_error.iloc[i, 1], size=24)
                        feats[start:end, 5] *= np.random.normal(WindOffShore_error.iloc[i, 0] + 1, WindOffShore_error.iloc[i, 1], size=24)
                        feats[start:end, 6] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)
                        feats[start:end, 7] *= np.random.normal(load_error.iloc[0, 0] + 1, load_error.iloc[1, 0], size=24)

                Xs.append(feats.flatten()) 
                Ys.append(y)

            return np.vstack(Xs), np.vstack(Ys)


        def build_model(input_dim, horizon, distribution):
            if distribution:
                if not mixture:
                    if Forecast_Horizon == 48:
                        (dropout, activation, layer_size, learning_and_weight, regularization) = ([0.1, 0.1], ['tanh', 'tanh'], [512, 256], [0.0001, 1e-04], [0.00001, 0.00001])
                    else:
                        (dropout, activation, layer_size, learning_and_weight, regularization) = ([0.1, 0.1], ['tanh', 'tanh'], [256, 128], [0.0001, 1e-04], [0.0001, 0.0001])
                else:
                    if Forecast_Horizon == 48:
                        (dropout, activation, layer_size, learning_and_weight, regularization) = ([0.25, 0.25], ['tanh', 'tanh'], [512, 256], [0.0001, 1e-04], [0.00001, 0.00001])
                    else:
                        (dropout, activation, layer_size, learning_and_weight, regularization) = ([0.1, 0.1], ['tanh', 'tanh'], [256, 128], [0.0001, 1e-04], [0.0001, 0.0001])   

            else:
                    if Forecast_Horizon == 48:
                        (dropout, activation, layer_size, learning_and_weight, regularization) = ([0.25, 0.1], ['tanh', 'tanh'], [256, 128], [0.0001, 1e-04], [0.0001, 0.0001])
                    else:
                        (dropout, activation, layer_size, learning_and_weight, regularization) = ([0.25, 0.1], ['tanh', 'tanh'], [256, 128], [0.0001, 1e-04], [0.0001, 0.0001])   

                

            
            inputs = keras.Input(shape=(input_dim,), name='input_layer')
            hidden = keras.layers.Dense(layer_size[0], 
                                    activation=activation[0], 
                                    kernel_regularizer=regularizers.l2(regularization[0]))(inputs)
            hidden = keras.layers.Dropout(dropout[0])(hidden)
            hidden = keras.layers.Dense(layer_size[1], 
                                    activation=activation[1], 
                                    kernel_regularizer=regularizers.l2(regularization[1]))(hidden)
            hidden = keras.layers.Dropout(dropout[1])(hidden)


            if not distribution:
                outputs = keras.layers.Dense(horizon, activation='linear')(hidden)
                model = keras.Model(inputs, outputs)
                model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_and_weight[0], weight_decay=learning_and_weight[1]),
                            loss='mae',
                            metrics=['mae'])
                return model

            else:
                if mixture:
                    param_output = keras.layers.Dense(9 * horizon)(hidden) 

                    def make_mixture_distribution(params):
                        params = tf.reshape(params, [-1, horizon, 9])

                        loc1 = params[:, :, 0]
                        scale1 = 1e-3 + 1.5 * tf.nn.softplus(params[:, :, 1])
                        tail1 = 1 + 1.5 * tf.nn.softplus(params[:, :, 2])
                        skew1 = params[:, :, 3]

                        loc2 = params[:, :, 4]
                        scale2 = 1e-3 + 1.5 * tf.nn.softplus(params[:, :, 5])
                        tail2 = 1 + 1.5 * tf.nn.softplus(params[:, :, 6])
                        skew2 = params[:, :, 7]

                        raw_logits = params[:, :, 8] 
                        logits = tf.stack([raw_logits, -raw_logits], axis=-1)  

                        locs = tf.stack([loc1, loc2], axis=-1)
                        scales = tf.stack([scale1, scale2], axis=-1)
                        tails = tf.stack([tail1, tail2], axis=-1)
                        skews = tf.stack([skew1, skew2], axis=-1)

                        components = tfd.JohnsonSU(
                            loc=locs,
                            scale=scales,
                            tailweight=tails,
                            skewness=skews
                        )

                        mixture_dist = tfd.MixtureSameFamily(
                            mixture_distribution=tfd.Categorical(logits=logits),
                            components_distribution=components
                        )

                        return mixture_dist

                    outputs = tfp.layers.DistributionLambda(make_mixture_distribution)(param_output)

                    model = keras.Model(inputs, outputs)
                    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_and_weight[0], weight_decay=learning_and_weight[1]),
                                loss=lambda y, rv_y: -rv_y.log_prob(y),
                                metrics=['mae'])
                    return model
                
                else:
                    param_layers = []
                    for p in range(4):
                        param_layers.append(keras.layers.Dense(
                            horizon, activation='linear')(hidden))
                    linear = tf.keras.layers.concatenate(param_layers)
                    outputs = tfp.layers.DistributionLambda(
                        lambda t: tfd.JohnsonSU(
                                loc=t[..., :horizon],
                                scale=1e-3 + 3 * tf.math.softplus(t[..., horizon:2*horizon]),
                                tailweight= 1 + 3 * tf.math.softplus(t[..., 2*horizon:3*horizon]),
                                skewness=t[..., 3*horizon:]))(linear)
                
                    model = keras.Model(inputs, outputs)

                    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_and_weight[0], weight_decay=learning_and_weight[1]), loss=lambda y, rv_y: -rv_y.log_prob(y),
                                metrics=['mae'])
                    return model


        def prepare_and_train(distribution):
            train_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/train_dates.csv', parse_dates=['date']).sort_values(by='date')
            val_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/val_dates.csv', parse_dates=['date']).sort_values(by='date')
            test_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/test_dates.csv', parse_dates=['date']).sort_values(by='date')
            train_dates = train_dates['date'].tolist()
            val_dates = val_dates['date'].tolist()
            test_dates = test_dates['date'].tolist()

            train_scaled, val_scaled, test_scaled, price_scaler = load_data(FILE_PATH)
            lookup_train = make_lookup(train_scaled)
            lookup_val = make_lookup(val_scaled)
            lookup_test = make_lookup(test_scaled)

            x_train, y_train = build_noisy_windows(lookup_train, train_dates, Forecast_Horizon, add_noise=True)
            x_val, y_val = build_noisy_windows(lookup_val, val_dates, Forecast_Horizon, add_noise=True)
            x_test, y_test= build_noisy_windows(lookup_test, test_dates, Forecast_Horizon, add_noise = True)

            model = build_model(input_dim=x_train.shape[1], horizon=Forecast_Horizon, distribution=distribution)

            train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(16)
            val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(16)
            test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(16)

            early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

            model.fit(train_ds, epochs=60, validation_data=val_ds, callbacks=[early_stop])

            if not distribution:
                preds = model.predict(test_ds)
                return preds, y_test, price_scaler, test_dates
            else:
                if not mixture:
                    def extract_distribution_parameters(model, test_ds, horizon):
                        locs = []
                        scales = []
                        tailweights = []
                        skewnesses = []

                        for x_batch, _ in test_ds:
                            dist = model(x_batch, training=False)
                            locs.append(dist.loc.numpy())
                            scales.append(dist.scale.numpy())
                            tailweights.append(dist.tailweight.numpy())
                            skewnesses.append(dist.skewness.numpy())

                        locs = np.concatenate(locs, axis=0)
                        scales = np.concatenate(scales, axis=0)
                        tailweights = np.concatenate(tailweights, axis=0)
                        skewnesses = np.concatenate(skewnesses, axis=0)

                        return locs, scales, tailweights, skewnesses
                    locs, scales, tailweights, skewnesses = extract_distribution_parameters(model, test_ds, horizon=Forecast_Horizon)
                    return locs, scales, tailweights, skewnesses, y_test, price_scaler, test_dates

                else:
                    def extract_distribution_parameters(model, test_ds, horizon):
                        locs = []
                        scales = []
                        tailweights = []
                        skewnesses = []
                        logits = []

                        for x_batch, _ in test_ds:
                            dist = model(x_batch, training=False)
                            locs.append(dist.components_distribution.loc.numpy())
                            scales.append(dist.components_distribution.scale.numpy())
                            tailweights.append(dist.components_distribution.tailweight.numpy())
                            skewnesses.append(dist.components_distribution.skewness.numpy())
                            logits.append(dist.mixture_distribution.logits.numpy())

                        locs = np.concatenate(locs, axis=0)
                        scales = np.concatenate(scales, axis=0)
                        tailweights = np.concatenate(tailweights, axis=0)
                        skewnesses = np.concatenate(skewnesses, axis=0)
                        logits = np.concatenate(logits, axis=0)
                        

                        return locs, scales, tailweights, skewnesses, logits
            
                    locs, scales, tailweights, skewnesses, logits = extract_distribution_parameters(model, test_ds, horizon=Forecast_Horizon)
                    return locs, scales, tailweights, skewnesses, logits, y_test, price_scaler, test_dates


        if not distribution:
            outputs, y_test, price_scaler, start_times = prepare_and_train(distribution)
            y_pred = price_scaler.inverse_transform(outputs)
            y_true = price_scaler.inverse_transform(y_test).round(2)
            rmse_val = calc_rmse(y_true, y_pred)
            all_preds.append(y_pred)


        else:
            if not mixture:
                locs, scales, tailweights, skewnesses, y_test, price_scaler, start_times = prepare_and_train(distribution)
                
                distributions = tfd.JohnsonSU(
                loc=np.array(locs),
                scale=np.array(scales),
                tailweight=np.array(tailweights),
                skewness=np.array(skewnesses)
            )
                
                n_samples = 2000
                samples = distributions.sample(n_samples) 
                samples_np = samples.numpy()

                quantiles = np.arange(0.01, 1.00, 0.01).round(2)
                samples_np = samples.numpy() 
                Y_pred = np.quantile(samples_np, q=quantiles, axis=0)  

                Y_pred = np.transpose(Y_pred, (1, 2, 0)) 
                (N, H, Q)  = Y_pred.shape

            
                Y_pred = price_scaler.inverse_transform(Y_pred.reshape(-1,1)).reshape(N, H, Q)
                y_true = price_scaler.inverse_transform(y_test).round(2)
                all_preds.append(Y_pred)
                
                quantiles = quantiles.tolist()
                midquantile = Y_pred[:, :, quantiles.index(0.5)]
                rmse_val = calc_rmse(y_true, midquantile)
                pinball_losses, pinball_loss_mean, crps = calculate_pinball_loss_CRPS(y_true, Y_pred, quantiles)
                nterval_scores, mean_interval_score = calculate_winkler(Y_pred, y_true, quantiles, lower_q = 0.2, upper_q=0.8)
   
            else:
                locs, scales, tailweights, skewnesses, logits, y_test, price_scaler, start_times = prepare_and_train(distribution)
                
                distributions = tfd.JohnsonSU(
                loc=np.array(locs),
                scale=np.array(scales),
                tailweight=np.array(tailweights),
                skewness=np.array(skewnesses),
            )
                mixture_distribution = tfd.MixtureSameFamily(
                    mixture_distribution=tfd.Categorical(logits=logits),
                    components_distribution=distributions,
                )
                
                
                n_samples = 2000
                samples = mixture_distribution.sample(n_samples) 
                quantiles = np.arange(0.01, 1.00, 0.01).round(2)
                samples_np = samples.numpy()  

                Y_pred = np.quantile(samples_np, q=quantiles, axis=0) 
                Y_pred = np.transpose(Y_pred, (1, 2, 0))  
                (N, H, q)  = Y_pred.shape

                Y_pred = price_scaler.inverse_transform(Y_pred.reshape(-1,1)).reshape(N, H, len(quantiles))
                all_preds.append(Y_pred)
                quantiles = quantiles.tolist()
                midquantile = Y_pred[:, :, quantiles.index(0.5)]
                y_true = price_scaler.inverse_transform(y_test).round(2)

                rmse_val = calc_rmse(y_true, midquantile)
                pinball_losses, pinball_loss_mean, crps = calculate_pinball_loss_CRPS(y_true, Y_pred, quantiles)
                interval_scores, mean_interval_score = calculate_winkler(Y_pred, y_true, quantiles, lower_q = 0.3, upper_q=0.7)

    all_preds_array = np.stack(all_preds, axis=0)
    mean_preds = np.mean(all_preds_array, axis=0)

    if distribution:
        pinball_losses, pinball_loss_mean, crps = calculate_pinball_loss_CRPS(y_true, mean_preds, quantiles)
        interval_scores, mean_interval_score = calculate_winkler(mean_preds, y_true, quantiles, lower_q = 0.3, upper_q=0.7)
        midquantile = mean_preds[:, :, quantiles.index(0.5)]
        avg_rmse = np.sqrt(mean_squared_error(y_true, midquantile))
        mean_preds = mean_preds.reshape(mean_preds.shape[0], -1)
    else:
        avg_rmse = np.sqrt(mean_squared_error(y_true, mean_preds))

    print(f"\n Avg_RMSE: {avg_rmse} \n")
    final_forecasts_df = pd.DataFrame(mean_preds)
    final_forecasts_df.to_csv(OUTPUT_PATH, index=False)


LQRA and LQRA Quick (Both tuning and forecasting), also used for ablation study



In [ ]:
import cplex
import numpy as np
import pandas as pd
from scipy.stats import norm
from numpy import log, sqrt, sinh
from datetime import timedelta
from tqdm import tqdm
from collections import defaultdict
import statsmodels.api as sm
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
import cvxpy as cp
from sklearn.metrics import mean_squared_error
import ecos
from joblib import Parallel, delayed

load_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/load_error.csv')
Solar_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/Solar_error.csv')
WindShore_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindShore_error.csv')
WindOffShore_error = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindOffShore_error.csv')

def calc_rmse(y_true, y_pred):
    rmse_val = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"RMSE: {rmse_val:.3f} €/MWh")
    return rmse_val

def pinball_loss(y_true, y_pred, quantiles):
   
    N, H, Q = y_pred.shape
    losses = np.zeros((N, H, Q))
    for q_idx, q in enumerate(quantiles):
        err = y_true - y_pred[:, :, q_idx]
        losses[:, :, q_idx] = np.maximum(q * err, (q - 1) * err)
    return losses

def calculate_pinball_loss_CRPS(y_true, Y_pred, quantiles):
    pinball_losses = pinball_loss(np.array(y_true), Y_pred, np.array(quantiles))
    print(f"Mean Pinball Loss: {np.mean(pinball_losses):.4f}")
    crps = np.mean(np.trapz(pinball_losses, np.array(quantiles), axis=-1))
    print(f'Approx CRPS: {crps:.4f}')  
    return pinball_losses, np.mean(pinball_losses), crps

def winkler_score(y_true, y_lower, y_upper, alpha):
    width = y_upper - y_lower
    under = (y_lower - y_true) * (y_true < y_lower)
    over  = (y_true - y_upper) * (y_true > y_upper)
    
    score = width + (2.0 / alpha) * (under + over)
    return score

def calculate_winkler(Y_pred, y_true, quantiles, lower_q, upper_q):
    alpha = 1.0 - (upper_q - lower_q)
    try:
        li = quantiles.index(lower_q)
        ui = quantiles.index(upper_q)
    except ValueError:
        raise ValueError(f"lower_q={lower_q} or upper_q={upper_q} not in quantiles list")
    
    y_lower = Y_pred[:, :, li]
    y_upper = Y_pred[:, :, ui]
    y_true = np.array(y_true)
    
    scores = winkler_score(y_true, y_lower, y_upper, alpha)
    mean_score = scores.mean()
    print(f"Mean Winkler score (α={alpha:.2f}, interval {lower_q}-{upper_q}): {mean_score:.4f}")
    return scores, mean_score

def robust_standardize(series):
    median = np.median(series)
    mad = np.median(np.abs(series - median))
    z_075 = 0.6745 
    scale = mad / z_075 if mad != 0 else 1e-6
    return (series - median) / scale, median, scale

def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['start_datetime', 'end_datetime'])

    df['hour'] = df['start_datetime'].dt.hour
    df['day'] = df['start_datetime'].dt.date
    df['dayofweek'] = df['start_datetime'].dt.dayofweek
    df['month'] = df['start_datetime'].dt.month
    df['is_sat'] = (df['dayofweek'] == 5).astype(int)
    df['is_sun'] = (df['dayofweek'] == 6).astype(int)
    df['is_mon'] = (df['dayofweek'] == 0).astype(int)

    categorical_features = ['hour', 'dayofweek', 'month', 'start_datetime', 'end_datetime', 'Timestamp', 'day', 'is_sat', 'is_sun', 'is_mon']
    features_to_scale = [col for col in df.columns if col not in categorical_features]

    train_df = df[(df['start_datetime'].dt.year == 2023)]
    val_df = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
    test_df  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]
    
    test_df_original = test_df.copy()

    for feature in features_to_scale:
        if feature == 'Price':
            x_std, x_median_price, x_scale_price = robust_standardize(train_df[feature])
            train_df.loc[:,feature] = np.arcsinh((train_df[feature] - x_median_price) / x_scale_price).copy()
            val_df.loc[:,feature] = np.arcsinh((val_df[feature] - x_median_price) / x_scale_price).copy()
            test_df.loc[:,feature] = np.arcsinh((test_df[feature] - x_median_price) / x_scale_price).copy()
            df.loc[:,feature] = np.arcsinh((df[feature] - x_median_price) / x_scale_price).copy()
        else:
            x_std, x_median, x_scale = robust_standardize(train_df[feature])
            train_df.loc[:,feature] = np.arcsinh((train_df[feature] - x_median) / x_scale).copy()
            val_df.loc[:,feature] = np.arcsinh((val_df[feature] - x_median) / x_scale).copy()
            test_df.loc[:,feature] = np.arcsinh((test_df[feature] - x_median) / x_scale).copy()
            df.loc[:,feature] = np.arcsinh((df[feature] - x_median) / x_scale).copy()

    df = pd.get_dummies(df, columns=['month'], drop_first=True)  

    return df, train_df, val_df, test_df, test_df_original, features_to_scale, x_median_price, x_scale_price


def make_lookup(df):
    from collections import defaultdict
    lookup = defaultdict(dict)
    for _, row in df.iterrows():
        lookup[(row['day'], row['hour'])] = row.to_dict()
    return lookup

def construct_features_day(lookup, df, base_day, horizon, add_noise=True):
    np.random.seed(int(base_day.strftime("%Y%m%d")))
    feats =[]
    for hour in range(horizon):
        days_offset = hour // 24
        h = hour % 24

        d = (base_day + timedelta(days=days_offset))
        d1 = (base_day - timedelta(days = 1))
        d2 = (base_day - timedelta(days = 2))
        d7 = (d - timedelta(days = 7))
        
      
        Z1, Z2, Z3, Z4, Z5, Z6 = [
            lookup.get((d, h), {}).get(name, np.nan)
            for name in ['Load', 'Solar', 'WindShore', 'WindOffShore', "Net_Import", "Coal_Price"]
            ]
        f = [
                lookup.get((d1, h), {}).get("Price", np.nan),
                lookup.get((d2, h), {}).get("Price", np.nan),
                lookup.get((d7, h), {}).get("Price", np.nan),
                lookup.get((d1, 23), {}).get("Price", np.nan),
                df[df['day']==(d1) ]['Price'].max(),
                df[df['day']==(d1) ]['Price'].min(),
                Z1, Z2, Z3, Z4, Z5, Z6,
                lookup.get((d, h),{}).get("is_sat", np.nan),
                lookup.get((d, h),{}).get("is_sun", np.nan),
                lookup.get((d, h),{}).get("is_mon", np.nan),
                *[lookup.get((d,h), {}).get(f'month_{m}', 0) for m in range(2, 13)]
         ]
       
        feats.append(f)
    feats = np.array(feats)
    if add_noise:
            start = 0
            while start < Forecast_Horizon:
                i = (start) // 24  
                end = start + 24
    
                feats[start:end, 6] *= np.random.normal(load_error.iloc[0,0]+ 1, load_error.iloc[1,0], size=24)
                feats[start:end, 8] *= np.random.normal(WindShore_error.iloc[i, 0] + 1, WindShore_error.iloc[i, 1], size=24)
                feats[start:end, 9] *= np.random.normal(WindOffShore_error.iloc[i,0] + 1, WindOffShore_error.iloc[i, 1], size=24)
                feats[start:end, 7] *= np.random.normal(Solar_error.iloc[i, 0] + 1, Solar_error.iloc[i, 1], size=24)
                feats[start:end, 10] *= np.random.normal(load_error.iloc[0,0]+ 1, load_error.iloc[1,0], size=24)
                feats[start:end, 11] *= np.random.normal(load_error.iloc[0,0]+ 1, load_error.iloc[1,0], size=24)

                start += 24
    return feats  

def cache_features_targets(df, days, lookup, horizon):
    features_day = {}
    targets_day = {}
    for base_day in tqdm(days, desc="Caching features and targets"):
        feats = construct_features_day(lookup, df, base_day, horizon)
        features_day[base_day] = feats  
        targets = np.array([
            lookup.get(((base_day + timedelta(days=i//24)), i%24),{}).get("Price", np.nan)
            for i in range(horizon)
        ])
        targets_day[base_day] = targets

    N = len(days)
    features_array = np.stack([features_day[d] for d in days])  
    targets_array = np.stack([targets_day[d] for d in days])   
    return features_array, targets_array


def forecast_day_calibration(
    features_array_all,
    targets_array_all,
    all_days,
    base_day,
    horizon,
    calibration_lengths,
    median, scale
):
    results = defaultdict(list)  

    forecast_day_idx = all_days.index(base_day)
    for h in range(horizon):
        preds_for_hour = []
        for cal_len in calibration_lengths:
            train_end_idx = forecast_day_idx
            train_start_idx = max(0, forecast_day_idx - cal_len)

            if train_start_idx == train_end_idx:
                preds_for_hour.append(None)
                continue

            X_train = features_array_all[train_start_idx:train_end_idx, h, :]
            y_train = targets_array_all[train_start_idx:train_end_idx, h]
            x_test = features_array_all[forecast_day_idx, h, :]

            X_train = sm.add_constant(X_train, has_constant='add')
            x_test = sm.add_constant(x_test.reshape(1, -1), has_constant='add')

            model = sm.OLS(y_train, X_train).fit()
            y_hat_transformed = model.predict(x_test)[0]

            residuals = np.asarray(y_train - model.fittedvalues) 

            pred_with_residuals = np.sinh(y_hat_transformed + residuals)  
            mean_pred = np.mean(pred_with_residuals)
            y_hat = median + scale * mean_pred

            preds_for_hour.append(y_hat)

        results[h] = preds_for_hour

    return base_day, results


def run_point_forecasting_parallel(
    features_array_all,
    targets_array_all,
    all_days,
    forecast_days,
    calibration_length,
    forecast_horizon,
    median,
    scale
):
    final_results = defaultdict(lambda: defaultdict(list))

    with ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(
                forecast_day_calibration,
                features_array_all,
                targets_array_all,
                all_days,
                fd,
                forecast_horizon,
                calibration_length,
                median,
                scale
            ): fd for fd in forecast_days
        }

        for future in tqdm(as_completed(futures), total=len(futures), desc="Parallel forecasting"):
            try:
                fd, day_results = future.result()
                for rel_hour, preds in day_results.items():
                    final_results[fd][rel_hour] = preds
            except Exception as e:
                print(f"Error for forecast day {futures[future]}: {e}")

    return final_results


def convert_results_to_array(results, forecast_horizon, calibration_lengths):
    test_days_sorted = sorted(results.keys())
    N_test_days = len(test_days_sorted)
    N_windows = len(calibration_lengths)

    forecasts_array = np.empty((N_test_days, forecast_horizon, N_windows))
    forecasts_array[:] = np.nan  

    for i, day in enumerate(test_days_sorted):
        for h in range(forecast_horizon):
            preds = results[day].get(h, [np.nan] * N_windows)
            if len(preds) != N_windows:
                padded_preds = np.full(N_windows, np.nan)
                padded_preds[:len(preds)] = preds
                preds = padded_preds
            forecasts_array[i, h, :] = preds

    return forecasts_array, test_days_sorted

def get_true_price_array(day_list, targets_array, unique_days, forecast_horizon):
    start_idx = unique_days.index(day_list[0].date())
    end_idx = start_idx + len(day_list)
    true_prices = targets_array[start_idx:end_idx]
    return np.array(true_prices) 


def compute_bic(y_true, y_pred, n_params, n_obs):
    residuals = y_true - y_pred
    loss = np.mean(np.abs(residuals)) 
    bic = n_obs * np.log(loss + 1e-8) + n_params * np.log(n_obs)
    return bic

def fit_lqra_quantile_bic(X, y, alpha, lambda_grid):
    best_bic = np.inf
    best_beta = None
    best_lambda = None
    n_obs, n_models = X.shape

    for lam in lambda_grid:
        beta = cp.Variable(n_models)
        residuals = y - X @ beta
        quantile_loss = cp.sum(cp.maximum(alpha * residuals, (alpha - 1) * residuals))
        l1_penalty = lam * cp.norm1(beta)
        problem = cp.Problem(cp.Minimize(quantile_loss + l1_penalty))
        problem.solve(solver=cp.CPLEX)

        if beta.value is not None:
            y_hat = X @ beta.value
            bic = compute_bic(y, y_hat, np.count_nonzero(beta.value), n_obs)
            if bic < best_bic:
                best_bic = bic
                best_beta = beta.value
                best_lambda = lam

    return best_beta, best_lambda


def rolling_lqra_bic_forecasting_hourly_lambda(
    forecast_array, true_array, quantile_levels, all_days, test_days, calibration_window
):

    n_total_days, horizon, n_models = forecast_array.shape
    n_quantiles = len(quantile_levels)
    lambda_grid = np.logspace(-1, 3, 19) #10 values for quick version
 
    Q_preds = np.full((len(test_days), horizon, n_quantiles), np.nan)
    all_param = np.empty((len(test_days), horizon, n_quantiles), dtype=object)
    lambda_per_h = np.full((horizon, n_quantiles), np.nan)  

    print("Tuning λ once for each hour (shared across quantiles)...")
    first_test_idx = all_days.index(test_days[0].date())
    train_start = first_test_idx - calibration_window
    train_end = first_test_idx

    for h in range(24):
        X_train = forecast_array[train_start:train_end, h, :]
        y_train = true_array[train_start:train_end, h]

        for q_idx, alpha in enumerate(tqdm(quantile_levels, desc=f"Tuning quantiles for hour {h}")):
            _, best_lam = fit_lqra_quantile_bic(X_train, y_train, alpha, lambda_grid)
            lambda_per_h[h, q_idx] = best_lam

    test_indices = [all_days.index(day.date()) for day in test_days]

    def process_day(i, test_idx):
        Q_pred_day = np.full((horizon, n_quantiles), np.nan)
        param_day = np.empty((horizon, n_quantiles), dtype=object)

        train_start = test_idx - calibration_window
        train_end = test_idx

        for hour in range(horizon):
            h = hour % 24
            X_train = forecast_array[train_start:train_end, hour, :]
            y_train = true_array[train_start:train_end, hour]
            X_test = forecast_array[test_idx, hour, :]

            results = [
                fit_lqra_quantile_bic(X_train, y_train, alpha, [lambda_per_h[h, q_idx]])
                for q_idx, alpha in enumerate(quantile_levels)
            ]

            for q_idx, (beta, _) in enumerate(results):
                if beta is not None:
                    Q_pred_day[hour, q_idx] = np.dot(X_test, beta)
                    param_day[hour, q_idx] = (beta, lambda_per_h[h, q_idx])

        return i, Q_pred_day, param_day

    print("Rolling LQRA forecasts (parallel)...")
    results = Parallel(n_jobs=-1)(
        delayed(process_day)(i, test_idx)
        for i, test_idx in enumerate(tqdm(test_indices))
    )

    for i, Q_pred_day, param_day in results:
        Q_preds[i] = Q_pred_day
        all_param[i] = param_day

    return Q_preds, all_param 



Forecast_Horizon = 48 # 48 or 144
FILE_PATH = 'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv'
train_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/train_dates.csv', parse_dates=['date']).sort_values(by='date')
val_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/val_dates.csv', parse_dates=['date']).sort_values(by='date')
test_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/test_dates.csv', parse_dates=['date']).sort_values(by='date')
train_dates = train_dates['date'].tolist()
val_dates = val_dates['date'].tolist()
test_dates = test_dates['date'].tolist()


df, train_df, val_df, test_df, test_df_original, features_to_scale, median, scale = load_data(FILE_PATH)
print(features_to_scale)
unique_days = sorted(df['start_datetime'].dt.date.unique())[7:]
calibration_lengths = range(50, 350, 15) #(50, 225, 15) for quick version
quantiles = np.arange(0.01, 1.00, 0.01).round(2).tolist()

lookup = make_lookup(df)
features_array, targets_array_scaled = cache_features_targets(df, unique_days, lookup, Forecast_Horizon)
y_true= get_true_price_array(test_dates, targets_array_scaled, unique_days, Forecast_Horizon)
y_true_std= np.sinh(y_true) 
y_true = y_true_std * scale + median

results = run_point_forecasting_parallel(features_array,targets_array_scaled, unique_days, unique_days, calibration_lengths,Forecast_Horizon, median, scale)

forecast_array, val_days_sorted = convert_results_to_array(results, Forecast_Horizon, list(calibration_lengths))

targets_array_std = np.sinh(targets_array_scaled) 
targets_array = targets_array_std* scale + median
Q_preds, all_params = rolling_lqra_bic_forecasting_hourly_lambda(
    forecast_array, targets_array, quantiles, unique_days, test_dates, calibration_window =364 #175 for quick version)
)
print(Q_preds.shape)
toSave = pd.DataFrame(Q_preds.reshape(Q_preds.shape[0], -1))
Forecast_Horizon_name = Forecast_Horizon + 24
toSave.to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/LQRA_prob_forecast_{Forecast_Horizon_name}.csv", index=False)

midquantile = Q_preds[:, :, quantiles.index(0.5)].round(2)

midquantile = pd.DataFrame(midquantile)
#midquantile.to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Attempt_LQRA_point_test_{Forecast_Horizon_name}.csv", index=False)

rmse_val = calc_rmse(y_true, midquantile)
pinball_losses, pinball_loss_mean, crps = calculate_pinball_loss_CRPS(y_true, Q_preds, quantiles)
interval_scores, mean_interval_score = calculate_winkler(Q_preds, y_true, quantiles, lower_q = 0.4, upper_q=0.6)




Generating true electricity prices in the right format for evaluation


In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

FILE_PATH = 'C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv'
df = pd.read_csv(FILE_PATH, parse_dates=['start_datetime', 'end_datetime'])

df['hour'] = df['start_datetime'].dt.hour
df['day'] = df['start_datetime'].dt.date

test_df  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]

test_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/test_dates.csv', parse_dates=['date']).sort_values(by='date')
test_dates = test_dates['date'].tolist()
test_dates = [d - timedelta(days=1) for d in test_dates] # Adjust to match the day in test_dates for h = 24, 72 or 168

def get_true_price_array(day_list, targets_array, forecast_horizon):
    true_prices = []
    for d in day_list:
        mask = targets_array['start_datetime'] == d
        start_idx = np.where(mask)[0][0]
        end_idx = start_idx + forecast_horizon
        daily_prices = targets_array[start_idx:end_idx]['Price'].values
        true_prices.append(daily_prices.reshape(-1))  
    return np.array(true_prices) 

for forecast_horizon in [24,72, 168]: #or 48, 144
    y_true = get_true_price_array(test_dates, test_df, forecast_horizon= forecast_horizon)
    y_true = pd.DataFrame(y_true, columns=[f'h+{i+1}' for i in range(forecast_horizon)])
    y_true.to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Full_true/Values_for_evaluation_{forecast_horizon}.csv", index=False)


Deterministic Optimisation

In [ ]:
import cplex 
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import cvxpy as cp
import numpy as np

E_max = 75           
range_km = 400       
c_max = 11          
eta = 0.95          
SoC_min = 0.2 * E_max        
SoC_max = 0.8 * E_max       
energy_per_km = E_max / range_km  

def get_trip_schedule(day_type, day_index=None):
    if day_type < 5:
        return [
            {"hour": 8, "duration": 9, "distance": 50, "prob": 1.0},
            {"hour": 18, "duration": 3, "distance": 30, "prob": 0.2}
        ]
    else:
        return [
            {"hour": 14, "duration": 4, "distance": 60, "prob": 0.5}
        ]
  

def create_availability_and_energy_matrix(day_of_week, horizon=24):
    availability = np.ones(horizon, dtype=bool) 
    energy = np.zeros(horizon) 
    trips = get_trip_schedule(day_of_week)

    for trip in trips:
        if np.random.rand() < trip.get("prob", 1.0):
            dep = trip["hour"]
            dur = trip["duration"]
            distance = trip["distance"]
            energy_trip = distance * energy_per_km  
            for h in range(dep, dep + dur):
                availability[h] = False
            energy[dep] = energy_trip
    return availability, energy

def get_trip_energy(day_type):
    trips = get_trip_schedule(day_type)
    energy = 0
    for trip in trips:
        if np.random.rand() < trip.get("prob", 1.0):
            energy += trip["distance"] * energy_per_km
    return energy

def get_night_charging_hours(forecasted, hours_to_select=3):
    night_charging_mask = np.zeros_like(forecasted, dtype=bool)
    num_days = len(forecasted) // 24

    for day in range(num_days):
        start = day * 24

        night_indices = []
        if day > 0:
            night_indices += [((day - 1) * 24 + h) for h in [22, 23]]
        night_indices += [(start + h) for h in range(7)]
        night_indices = [i for i in night_indices if i < len(forecasted)]

        night_prices = [(i, forecasted[i]) for i in night_indices]
        lowest_hours = sorted(night_prices, key=lambda x: x[1])[:hours_to_select]

        for idx, _ in lowest_hours:
            night_charging_mask[idx] = True

    return night_charging_mask

def simulate_strategy(strategy, availability, trip_energy, State_of_Charge, Charging, Y_true):
    charging_strategy = np.zeros_like(Y_true)
    SoC = np.zeros_like(Y_true)
    State = State_of_Charge 
    charging = Charging
    night_charging_mask = get_night_charging_hours(Y_true)
   
    for t in range(len(Y_true)):
        if not availability[t]:  
            charging = 0  
            State = State - trip_energy[t]  
            if State < SoC_min:
                print(f"Warning: SoC below minimum at time {t}, current SoC: {State} on day {t} for strategy = {strategy}.")

        can_charge = availability[t]

        if strategy == "default" and can_charge:
            if State < SoC_max:
                charging = min(c_max, (SoC_max - State)/eta)  
                State = State + charging * eta 
            else:
                charging = 0  

        elif strategy == "rule" and can_charge: 
            if State < 0.5 * SoC_max or charging != 0:  
                charging = min(c_max, (SoC_max - State)/eta)  
                State = State + charging * eta  
            else:
                charging = 0
        
        elif strategy == "night" and can_charge:
            if night_charging_mask[t] and State < SoC_max:
                charging = min(c_max, (SoC_max - State) / eta)
                State += charging * eta
            else:
                charging = 0

        charging_strategy[t] = charging
        SoC[t]= State

    return charging_strategy, SoC

def compute_cost(Charging_Strategy, prices):
    prices_in_KWh_excl_tax = prices / 1000  
    prices_in_KWh_incl_tax = prices/1000 +  0.10154
    return np.sum(Charging_Strategy * prices_in_KWh_excl_tax), np.sum(Charging_Strategy*prices_in_KWh_incl_tax)


def load_and_prepare_data(Forecast_Horizon, model, true_values_24):
    horizon_file = Forecast_Horizon -24
    y_pred = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/{model}_forecasts_{horizon_file}.csv")
    Y_pred = pd.concat([true_values_24, y_pred], axis=1)
    return Y_pred

def load_and_prepare_quantile_data(quantile, probability_forecasts, true_values_24, all_quantiles):
    quantile_pred = pd.DataFrame(probability_forecasts[:, :, all_quantiles.index(quantile)])
    Y_pred = pd.concat([true_values_24, quantile_pred], axis=1)
    return Y_pred

def create_availability_and_energy_matrices(test_dates, Forecast_Horizon,  create_matrices):
    if create_matrices:
    
        Y_available = []
        Y_energy_trip = []
        extra = test_dates.iloc[-7:].apply(lambda d: d + timedelta(days=7))

        extra_days = pd.concat([test_dates, extra], ignore_index=True)

        for start_day in extra_days:
            availability, energy = create_availability_and_energy_matrix(day_of_week=start_day.dayofweek, horizon=24)
            Y_available.append(pd.DataFrame([[start_day] + availability.tolist()]))
            Y_energy_trip.append(pd.DataFrame([[start_day] + energy.tolist()]))

        Y_available_df = pd.concat(Y_available, ignore_index=True)
        Y_available_df.columns = ['start_datetime'] + [f"t+{i}" for i in range(24)]
        Y_energy_trip_df = pd.concat(Y_energy_trip, ignore_index=True)
        Y_energy_trip_df.columns = ['start_datetime'] + [f"t+{i}" for i in range(24)]

        for horizon in [24, 72, 168]:
            print(f"Processing horizon: {horizon}")
            Y_available = []
            Y_energy_trip = []

            for start_day in test_dates:
                avail =[]
                ener =[]
                for i in range(0, 1 if horizon == 24 else (3 if horizon == 72 else 7)): 
                    next_day = start_day + timedelta(days=i)
                    availability = Y_available_df[Y_available_df['start_datetime'] == next_day].iloc[:, 1:].values[0]
                    avail.extend(availability)
                    energy = Y_energy_trip_df[Y_energy_trip_df['start_datetime'] == next_day].iloc[:, 1:].values[0]
                    ener.extend(energy)

                Y_available.append(pd.DataFrame([[start_day] + avail]))
                Y_energy_trip.append(pd.DataFrame([[start_day] + ener]))
    
            Y_available = pd.concat(Y_available, ignore_index=True)
            Y_available.columns = ['start_datetime'] + [f"t+{i}" for i in range(horizon)]
            Y_energy_trip = pd.concat(Y_energy_trip, ignore_index=True)
            Y_energy_trip.columns = ['start_datetime'] + [f"t+{i}" for i in range(horizon)]
            Y_available.to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/Y_available_{horizon}.csv", index=False)
            Y_energy_trip.to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/Y_energy_trip_{horizon}.csv", index=False) 
    else:
        Y_available = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/Y_available_{Forecast_Horizon}.csv", parse_dates=['start_datetime'])
        Y_energy_trip = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/Y_energy_trip_{Forecast_Horizon}.csv", parse_dates=['start_datetime'])

    return Y_available, Y_energy_trip

def load_all_benchmark_strategies(strategies, Forecast_Horizon, Y_true, Y_available, Y_energy_trip):
    results = []
    State = SoC_max
    charging = 0

    for strategy in strategies:
        full_charging_strategy = []
        full_soc = []
        for i in tqdm(range(Y_available.shape[0])):  
            true_prices = Y_true.iloc[i,].values[:Forecast_Horizon]
            availability = Y_available.iloc[i,1:].values[:Forecast_Horizon].astype(bool)
            energy_trip = Y_energy_trip.iloc[i,1:].values[:Forecast_Horizon]

            charging_strategy, SoC = simulate_strategy(strategy=strategy, availability=availability, trip_energy=energy_trip, State_of_Charge=State, Charging=charging, Y_true = true_prices)
            State = SoC[23]
            charging = charging_strategy[23]

            full_charging_strategy.append(charging_strategy)
            full_soc.append(SoC)
                
            cost_excl_tax, cost_incl_tax = compute_cost(charging_strategy[:24], true_prices[:24])

            results.append({
                "sample": i,
                "strategy": strategy,
                "cost_excl_tax": cost_excl_tax,
                "cost_incl_tax": cost_incl_tax
                })

        pd.DataFrame(full_charging_strategy).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/Charging/{strategy}_strategy_charge_schedule.csv")
        pd.DataFrame(full_soc).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/SoC/{strategy}_strategy_SoC.csv")

    results_df = pd.DataFrame(results)
    results_df = pd.DataFrame(results)
    summary = results_df.groupby("strategy").agg(
    total_cost_excl_tax=("cost_excl_tax", "sum"),
    daily_mean_excl_tax=("cost_excl_tax", "mean"),
    daily_std_excl_tax=("cost_excl_tax", "std"),
    total_cost_incl_tax=("cost_incl_tax", "sum"),
    daily_mean_incl_tax=("cost_incl_tax", "mean"),
    daily_std_incl_tax=("cost_incl_tax", "std"),
    )
    return results_df, summary


def optimize_ev_charging(P, availability, trip_energy, SoC_init):
    H = len(P)  
    P = P/1000 

    charging = cp.Variable(H)
    soc = cp.Variable(H + 1)  

    constraints = [soc[0] == SoC_init]

    for t in range(H):
        if not availability[t]:
            constraints += [charging[t] == 0]
        else:
            constraints += [
                charging[t] >= 0,
                charging[t] <= c_max,  
            ]

        constraints += [
            soc[t] >= SoC_min,
            soc[t] <= SoC_max,
        ]

        if trip_energy[t] > 0:
            constraints += [
                soc[t] >= trip_energy[t] + SoC_min,
                soc[t+1] == soc[t] - trip_energy[t]
            ]
        else:
            constraints += [
                soc[t+1] == soc[t] + eta * charging[t]]

    objective = cp.Minimize(cp.sum(cp.multiply(P, charging)))
    problem = cp.Problem(objective, constraints)
    problem.solve(solver = cp.CPLEX)

    if problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"Optimization failed with status: {problem.status}")
        return None, None

    soc.value = np.round(soc.value, decimals=3)
    charging.value[np.abs(charging.value) < 1e-3] = 0

    return charging.value , soc.value[1:]

def optimize_ev_charging_twostage(P_prob, availability, trip_energy, SoC_init):
    import cvxpy as cp
    import numpy as np

    P_prob = P_prob / 1000  

    H = len(P_prob)
    charging_1 = cp.Variable(24)
    soc_1 = cp.Variable(25)

    hours_stage2 = H - 24
    num_days_stage2 = hours_stage2 // 24
    charging_shared = [cp.Variable(24) for _ in range(num_days_stage2)]
    soc_shared = [cp.Variable(25) for _ in range(num_days_stage2)]

    constraints = []
    constraints.append(soc_1[0] == SoC_init)

    for t in range(24):
        if not availability[t]:
            constraints.append(charging_1[t] == 0)
        else:
            constraints += [0 <= charging_1[t], charging_1[t] <= c_max]
        constraints += [SoC_min <= soc_1[t], soc_1[t] <= SoC_max]
        if trip_energy[t] > 0:
            constraints += [
                soc_1[t] >= trip_energy[t] + SoC_min,
                soc_1[t + 1] == soc_1[t] - trip_energy[t]
            ]
        else:
            constraints.append(soc_1[t + 1] == soc_1[t] + eta * charging_1[t])

    constraints.append(soc_shared[0][0] == soc_1[24])

    for d in range(num_days_stage2):
        c = charging_shared[d]
        s = soc_shared[d]
        t_global = 24 + d * 24  

        if d > 0:
            constraints.append(s[0] == soc_shared[d - 1][24])  

        for h in range(24):
            t_abs = t_global + h
            if not availability[t_abs]:
                constraints.append(c[h] == 0)
            else:
                constraints += [0 <= c[h], c[h] <= c_max]

            constraints += [SoC_min <= s[h], s[h] <= SoC_max]

            if trip_energy[t_abs] > 0:
                constraints += [
                    s[h] >= trip_energy[t_abs] + SoC_min,
                    s[h + 1] == s[h] - trip_energy[t_abs]
                ]
            else:
                constraints.append(s[h + 1] == s[h] + eta * c[h])

    cost_day1 = cp.sum(cp.multiply(P_prob[:24], charging_1))

    cost_stage2 = 0
    for d in range(num_days_stage2):
        P_day_d = P_prob[(d+1) * 24:(d + 2) * 24] 
        cost_stage2 += cp.sum(cp.multiply(P_day_d, charging_shared[d]))

    total_cost = cost_day1 + cost_stage2

    problem = cp.Problem(cp.Minimize(total_cost), constraints)
    problem.solve(solver=cp.CPLEX, verbose=False)

    if problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"Optimisation failed: {problem.status}")
        return None, None

    charging_total = [charging_1.value] + [c.value for c in charging_shared]
    soc_total = [soc_1.value[1:]] + [s.value[1:] for s in soc_shared]

    charging_total = np.concatenate(charging_total)
    soc_total = np.concatenate(soc_total)

    charging_total[np.abs(charging_total) < 1e-3] = 0
    soc_total = np.round(soc_total, 3)

    return charging_total, soc_total


def load_optimization_point_forecasts(Forecast_Horizon, Y_true, Y_pred, Y_available, Y_energy_trip, model):
    State = SoC_max
    charging = 0
    Strategy_optimization = []
    SoC_optimization = []
    results = []
    for i in tqdm(range(Y_pred.shape[0])): 
        forecast_prices = Y_pred.iloc[i,].values[:Forecast_Horizon]
        true_prices = Y_true.iloc[i, ].values[:Forecast_Horizon]
        availability = Y_available.iloc[i, 1:].values[:Forecast_Horizon].astype(bool)
        energy_trip = Y_energy_trip.iloc[i, 1:].values[:Forecast_Horizon]

        if Forecast_Horizon > 24:    
            charging_strategy, SoC = optimize_ev_charging_twostage(forecast_prices, availability, energy_trip, State)
        else:
            charging_strategy, SoC = optimize_ev_charging(forecast_prices, availability, energy_trip, State)
        State = SoC[23]
        Strategy_optimization.append(charging_strategy)
        SoC_optimization.append(SoC)

        cost_excl_tax, cost_incl_tax = compute_cost(charging_strategy[:24], true_prices[:24])

        results.append({
                    "sample": i,
                    "strategy": model,
                    "horizon": Forecast_Horizon,
                    "cost_excl_tax": cost_excl_tax,
                    "cost_incl_tax": cost_incl_tax
                    })


    results_df = pd.DataFrame(results)
    summary = results_df.groupby("strategy").agg(
    total_cost_excl_tax=("cost_excl_tax", "sum"),
    daily_mean_excl_tax=("cost_excl_tax", "mean"),
    daily_std_excl_tax=("cost_excl_tax", "std"),
    total_cost_incl_tax=("cost_incl_tax", "sum"),
    daily_mean_incl_tax=("cost_incl_tax", "mean"),
    daily_std_incl_tax=("cost_incl_tax", "std"),
    )
    pd.DataFrame(Strategy_optimization).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/Charging/{model}_charge_schedule_{Forecast_Horizon}.csv")
    pd.DataFrame(SoC_optimization).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/SoC/{model}_SoC_{Forecast_Horizon}.csv")
    return results_df, summary


true_values_24 = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Full_true/True_values_24.csv")
test_dates = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/test_dates.csv', parse_dates=['date']).sort_values(by='date')
test_dates = test_dates['date']
test_dates = [d - timedelta(days=1) for d in test_dates] 
test_dates = pd.Series(test_dates, name='start_datetime')

create_availability_and_energy_matrices(test_dates, Forecast_Horizon = None, create_matrices = True)
Y_available24 = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/Y_available_24.csv", parse_dates=['start_datetime'])
Y_energy_trip24 = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/Y_energy_trip_24.csv", parse_dates=['start_datetime'])
strategies = ["default", "rule", "night"] 
results_strategy, summary_strategy = load_all_benchmark_strategies(strategies, Forecast_Horizon = 24, Y_true=true_values_24, Y_available = Y_available24, Y_energy_trip = Y_energy_trip24)
results24, summary24 = load_optimization_point_forecasts(Forecast_Horizon=24, Y_true = true_values_24, Y_pred=true_values_24, Y_available= Y_available24, Y_energy_trip = Y_energy_trip24, model ="24hours")
pd.concat([results_strategy, results24], ignore_index=False).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/Benchmarks_charging_cost_eachday.csv")
pd.concat([summary_strategy, summary24], ignore_index=False).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/Benchmarks_charging_cost_average.csv")
print(pd.concat([summary_strategy, summary24], ignore_index=False))

for Forecast_Horizon in [72, 168]: 
    all_result = []
    all_summary = []
    true_values = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Full_true/True_values_{Forecast_Horizon}.csv")
    Y_available, Y_energy_trip = create_availability_and_energy_matrices(test_dates, Forecast_Horizon=Forecast_Horizon, create_matrices = False)
    list_names = ['Arima', 'GJR_Garch', 'Garch', 'Hybrid_LSTM_lgbm', 'Hybrid_LSTM_xgb', "Hybrid_weighted_AllThree", 'Hybrid_weighted_LSTM_lgbm', 'Hybrid_weighted_LSTM_xgb', "LGBM", 'LSTM', "XGB", "DNN", "Full_info"]
    
    for model in list_names:
        if model == 'Full_info':
           Y_pred = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Full_true/True_values_{Forecast_Horizon}.csv")
        else:
            Y_pred = load_and_prepare_data(Forecast_Horizon, model, true_values_24)
        result, summary = load_optimization_point_forecasts(Forecast_Horizon, true_values, Y_pred, Y_available, Y_energy_trip, model)
        all_result.append(result)
        all_summary.append(summary)
    pd.concat(all_result, ignore_index=False).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/Optimization_charging_cost_eachday_{Forecast_Horizon}.csv")
    pd.concat(all_summary, ignore_index=False).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/Optimization_charging_cost_average_{Forecast_Horizon}.csv")
    print(pd.concat(all_summary, ignore_index=False))
    
    model_names = ['DDNN', 'DDNN_mixture', 'Regular_LQRA', 'Very_quick_LQRA']
    for name in  model_names:
        horizon = Forecast_Horizon - 24
        prob = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Probability/{name}_prob_forecasts_{horizon}.csv").values.reshape(290, horizon, 99)
        all_results = []
        all_summary = []
        all_quantiles = np.arange(0.01, 1.00, 0.01).round(2).tolist()
        for quantile in [0.5]:
            y_pred_quantile = load_and_prepare_quantile_data(quantile, prob, true_values_24, all_quantiles)
            result, summary = load_optimization_point_forecasts(Forecast_Horizon, true_values, y_pred_quantile, Y_available, Y_energy_trip, f"{name}_{quantile}")
            all_results.append(result)
            all_summary.append(summary)
        pd.concat(all_results, ignore_index=False).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/{name}_quantiles_charging_cost_eachday_{Forecast_Horizon}.csv")
        pd.concat(all_summary, ignore_index=False).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/{name}_quantiles_charging_cost_average_{Forecast_Horizon}.csv")
        print(pd.concat(all_summary, ignore_index=False))



Stochastic optimisation

In [ ]:
import cplex
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import cvxpy as cp
import numpy as np
from scipy.stats import norm

E_max = 75          
range_km = 400     
c_max = 11          
eta = 0.95         
SoC_min = 0.2 * E_max       
SoC_max = 0.8 * E_max       
energy_per_km = E_max / range_km  


def compute_cost(Charging_Strategy, prices):
    prices_in_KWh_excl_tax = prices / 1000  
    prices_in_KWh_incl_tax = prices/1000 + 0.10154
    return np.sum(Charging_Strategy * prices_in_KWh_excl_tax), np.sum(Charging_Strategy*prices_in_KWh_incl_tax)


def load_availability_and_energy_matrices(Forecast_Horizon):
    Y_available = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/Y_available_{Forecast_Horizon}.csv", parse_dates=['start_datetime'])
    Y_energy_trip = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/Y_energy_trip_{Forecast_Horizon}.csv", parse_dates=['start_datetime'])
    return Y_available, Y_energy_trip


def optimize_ev_charging_stochastic(Forecast_24h, Forecast_prob, availability, trip_energy, SoC_init, weights, Forecast_Horizon):
    Forecast_24h = Forecast_24h / 1000  
    Forecast_prob = Forecast_prob / 1000

    Q = len(weights)  
    T = Forecast_Horizon 
    W = weights

    charging = cp.Variable(T)       
    soc = cp.Variable(T + 1)         

    constraints = [soc[0] == SoC_init]

    for t in range(T):
        if not availability[t]:
            constraints.append(charging[t] == 0)
        else:
            constraints.append(charging[t] >= 0)
            constraints.append(charging[t] <= c_max)

        constraints.append(soc[t] >= SoC_min)
        constraints.append(soc[t] <= SoC_max)

        if trip_energy[t] > 0:
            constraints.append(soc[t] >= trip_energy[t] + SoC_min)
            constraints.append(soc[t + 1] == soc[t] - trip_energy[t])
        else:
            constraints.append(soc[t + 1] == soc[t] + eta * charging[t])

    cost_day1 = cp.sum(cp.multiply(Forecast_24h, charging[:24]))

    cost_after24h = 0
    for day in range((Forecast_Horizon-24)//24):
        for q in range(Q):
            scenario_prices = Forecast_prob[day*24:(day+1)*24, q]
            cost_q = cp.sum(cp.multiply(scenario_prices, charging[(day+1)*24:(day+2)*24])) 
            cost_after24h += W[q] * cost_q
    
    total_cost = cost_day1 + cost_after24h
    problem = cp.Problem(cp.Minimize(total_cost), constraints)
    problem.solve(solver=cp.CPLEX)

    if problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"Optimization failed with status: {problem.status}")
        return None, None

    charging_opt = charging.value
    soc_opt = soc.value[1:] 

    charging_opt[np.abs(charging_opt) < 1e-3] = 0
    soc_opt = np.round(soc_opt, decimals=3)

    return charging_opt, soc_opt


def load_optimization_prob_forecasts(Y_pred_prob, true_values_24, true_values, weights, Forecast_Horizon, model):
    State = SoC_max
    charging = 0
    Strategy_optimization = []
    SoC_optimization = []
    results = []
    for i in tqdm(range(true_values.shape[0])): 
        forecast_prices_24h = true_values_24.iloc[i, ].values[:24]
        forecast_prices_prob = Y_pred_prob[i, :Forecast_Horizon-24, :]  
        true_prices = true_values.iloc[i, ].values[:Forecast_Horizon]
        availability = Y_available.iloc[i, 1:].values[:Forecast_Horizon].astype(bool)
        energy_trip = Y_energy_trip.iloc[i, 1:].values[:Forecast_Horizon]
        
       
        charging_strategy, SoC = optimize_ev_charging_stochastic(forecast_prices_24h, forecast_prices_prob, availability, energy_trip, State, weights, Forecast_Horizon)

        State = SoC[23]
        Strategy_optimization.append(charging_strategy)
        SoC_optimization.append(SoC)

        cost_excl_tax, cost_incl_tax = compute_cost(charging_strategy[:24], true_prices[:24])
    

        results.append({"sample": i,
                    "strategy": f'{model}',
                    "cost_excl_tax": cost_excl_tax,
                    "cost_incl_tax": cost_incl_tax
                    })
        

    results_df = pd.DataFrame(results)
    summary = results_df.groupby("strategy").agg(
    total_cost_excl_tax=("cost_excl_tax", "sum"),
    daily_mean_excl_tax=("cost_excl_tax", "mean"),
    daily_std_excl_tax=("cost_excl_tax", "std"),
    total_cost_incl_tax=("cost_incl_tax", "sum"),
    daily_mean_incl_tax=("cost_incl_tax", "mean"),
    daily_std_incl_tax=("cost_incl_tax", "std"),
    )
    pd.DataFrame(Strategy_optimization).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/Charging/{model}_prob_charge_schedule_{Forecast_Horizon}.csv")
    pd.DataFrame(SoC_optimization).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Schedules/SoC/{model}_prob_SoC_{Forecast_Horizon}.csv")
    return results_df, summary


for Forecast_Horizon in [72, 168]:
    true_values_24 = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Full_true/True_values_24.csv")
    true_values = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Full_true/True_values_{Forecast_Horizon}.csv")
    all_results = []
    all_summary = []
    model_names = ["DDNN", "DDNN_mixture", "Regular_LQRA", "Quick_LQRA"]
    for model in model_names:
        horizon = Forecast_Horizon - 24
        Y_pred_prob = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Probability/{model}_prob_forecasts_{horizon}.csv").values.reshape(290, Forecast_Horizon-24, 99)
        Y_available, Y_energy_trip = load_availability_and_energy_matrices(Forecast_Horizon)
        Q = 99
        weights = np.ones(Q)/Q
        results_optimization, summary_optimization = load_optimization_prob_forecasts(Y_pred_prob, true_values_24, true_values, weights, Forecast_Horizon, model)
        all_results.append(results_optimization)
        all_summary.append(summary_optimization)
    pd.concat(all_results, ignore_index=False).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/Probability_charging_cost_eachday{Forecast_Horizon}.csv")
    pd.concat(all_summary, ignore_index=False).to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/Proabability_charging_cost_average_{Forecast_Horizon}.csv")
    print(pd.concat(all_summary, ignore_index=False))
        


Generate RMSE, CRPS and Winkler Scores

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

def calc_rmse(y_true, y_pred):
    rmse_val = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse_val

def pinball_loss(y_true, y_pred, quantiles):
   
    N, H, Q = y_pred.shape
    losses = np.zeros((N, H, Q))
    for q_idx, q in enumerate(quantiles):
        err = y_true - y_pred[:, :, q_idx]
        losses[:, :, q_idx] = np.maximum(q * err, (q - 1) * err)
    return losses

def calculate_pinball_loss_CRPS(y_true, Y_pred, quantiles):
    pinball_losses = pinball_loss(np.array(y_true), Y_pred, np.array(quantiles))
    crps = np.mean(np.trapz(pinball_losses, np.array(quantiles), axis=-1))
    return pinball_losses, np.mean(pinball_losses), crps

def winkler_score(y_true, y_lower, y_upper, alpha):
    width = y_upper - y_lower
    
    under = (y_lower - y_true) * (y_true < y_lower)
    over  = (y_true - y_upper) * (y_true > y_upper)
    
    score = width + (2.0 / alpha) * (under + over)
    return score

def calculate_winkler(Y_pred, y_true, quantiles, lower_q, upper_q):
    alpha = 1.0 - (upper_q - lower_q)
    try:
        li = quantiles.index(lower_q)
        ui = quantiles.index(upper_q)
    except ValueError:
        raise ValueError(f"lower_q={lower_q} or upper_q={upper_q} not in quantiles list")
    
    y_lower = Y_pred[:, :, li]
    y_upper = Y_pred[:, :, ui]
    y_true = np.array(y_true)
    
    scores = winkler_score(y_true, y_lower, y_upper, alpha)
    mean_score = scores.mean()
    return scores, mean_score

def load_and_prepare_quantile_data(quantile, probability_forecasts, all_quantiles):
    quantile_pred = pd.DataFrame(probability_forecasts[:, :, all_quantiles.index(quantile)])
    return quantile_pred

for Forecast_Horizon in [48, 144]: 
    results = []
    true_values = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Full_true/Values_for_evaluation_{Forecast_Horizon}.csv")
    list_names = ['Arima', 'garch_asymmetric', 'garch_regular', 'Hybrid_LSTM_lgbm', 'Hybrid_LSTM_xgb', "Hybrid_weighted_AllThree", 'Hybrid_weighted_LSTM_lgbm', 'Hybrid_weighted_LSTM_xgb', "LGBM", 'LSTM', "XGB", 'DNN']
    for model in list_names:
        y_pred = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/{model}_forecasts_{Forecast_Horizon}.csv")
        rmse = calc_rmse(true_values, y_pred)
        results.append((model, rmse))
    
    prob_results = []
    all_quantiles = np.arange(0.01, 1.00, 0.01).round(2).tolist()
    model_names = ["Regular_LQRA", "Quick_LQRA", "DDNN", "DDNN_mixture"]
    for prob_model in model_names:
        prob_pred = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Probability/{prob_model}_prob_forecasts_{Forecast_Horizon}.csv").values.reshape(290, Forecast_Horizon, 99)
        pinball_losses, avg_pinball_loss, crps = calculate_pinball_loss_CRPS(true_values, prob_pred, all_quantiles)   
        scores, mean_score_1 = calculate_winkler(prob_pred, true_values, all_quantiles, 0.4, 0.6)
        scores, mean_score_2 = calculate_winkler(prob_pred, true_values, all_quantiles, 0.3, 0.7)
        scores, mean_score_3 = calculate_winkler(prob_pred, true_values, all_quantiles, 0.2, 0.8)
        scores, mean_score_4 = calculate_winkler(prob_pred, true_values, all_quantiles, 0.1, 0.9)
        scores, mean_score_5 = calculate_winkler(prob_pred, true_values, all_quantiles, 0.05, 0.95)
        prob_results.append((prob_model, avg_pinball_loss, crps, mean_score_1, mean_score_2, mean_score_3, mean_score_4, mean_score_5))
        for quantile in np.arange(0.1, 1.0, 0.1).round(2):
            y_pred_quantile = load_and_prepare_quantile_data(quantile, prob_pred, all_quantiles)
            rmse = calc_rmse(true_values, y_pred_quantile)
            results.append((f"{prob_model}_Q_{quantile}", rmse))
            
    results_df = pd.DataFrame(results)
    results_df.columns = ["Model", "RMSE"]
    prob_results_df = pd.DataFrame(prob_results)
    prob_results_df.columns = ["Model", "Avg_pinball_loss", "CRPS", "Winkler_Score(40%-60%)", "Winkler_Score(30%-70%)", "Winkler_Score(20%-80%)", "Winkler_Score(10%-90%)", "Winkler_Score(5%-95%)"]
    results_df.to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/Point_pred_RMSE_{Forecast_Horizon}_extra.csv", index=False )
    prob_results_df.to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/Prob_pred_CRPS_Winkler_{Forecast_Horizon}.csv", index=False)
  


Generate evaluation metrics for ablation study

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error

def calc_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def pinball_loss(y_true, y_pred, quantiles):
    N, H, Q = y_pred.shape
    losses = np.zeros((N, H, Q))
    for q_idx, q in enumerate(quantiles):
        err = y_true - y_pred[:, :, q_idx]
        losses[:, :, q_idx] = np.maximum(q * err, (q - 1) * err)
    return losses

def calculate_pinball_loss_CRPS(y_true, Y_pred, quantiles):
    pinball_losses = pinball_loss(np.array(y_true), Y_pred, np.array(quantiles))
    crps = np.mean(np.trapz(pinball_losses, np.array(quantiles), axis=-1))
    return crps

def winkler_score(y_true, y_lower, y_upper, alpha):
    width = y_upper - y_lower
    under = (y_lower - y_true) * (y_true < y_lower)
    over = (y_true - y_upper) * (y_true > y_upper)
    score = width + (2.0 / alpha) * (under + over)
    return score

def calculate_winkler(Y_pred, y_true, quantiles, lower_q, upper_q):
    alpha = 1.0 - (upper_q - lower_q)
    li = quantiles.index(lower_q)
    ui = quantiles.index(upper_q)
    y_lower = Y_pred[:, :, li]
    y_upper = Y_pred[:, :, ui]
    return np.mean(winkler_score(np.array(y_true), y_lower, y_upper, alpha))

def calculate_interval_width(Y_pred, quantiles, lower_q, upper_q):
    li = quantiles.index(lower_q)
    ui = quantiles.index(upper_q)
    y_lower = Y_pred[:, :, li]
    y_upper = Y_pred[:, :, ui]
    return np.mean(y_upper - y_lower)

def coverage_probability(y_true, y_pred, lower_idx, upper_idx):
    y_lower = y_pred[:, :, lower_idx]
    y_upper = y_pred[:, :, upper_idx]
    inside = (y_true >= y_lower) & (y_true <= y_upper)
    coverage = np.mean(inside)
    return coverage

ablation_vars = ["Quick", "No_Solar", "No_Wind", "No_Load", "No_Last_Day", "No_d7"]
quantiles = np.round(np.arange(0.01, 1.00, 0.01), 2).tolist()

results = []
for h in [72, 168]:
    y_true = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Full_true/Values_for_evaluation_{h - 24}.csv").values
    for var in ablation_vars:
        file_path = f"C:/Users/daniq/iCloudDrive\Erasmus University Rotterdam/Master/Thesis/Output/{var}_LQRA_prob_test_{h}.csv"
        if not os.path.exists(file_path):
            print(f"Missing: {file_path}")
            continue
        y_pred = pd.read_csv(file_path).values.reshape(290, h - 24, 99)

        median_idx = quantiles.index(0.5)
        y_median = y_pred[:, :, median_idx]
        rmse = calc_rmse(y_true, y_median)

        crps = calculate_pinball_loss_CRPS(y_true, y_pred, quantiles)
        winkler_80 = calculate_winkler(y_pred, y_true, quantiles, 0.1, 0.9)
        interval_width = np.mean(y_pred[:, :, quantiles.index(0.9)] - y_pred[:, :, quantiles.index(0.1)])

        width_by_segment = []
        for i in range(0, h - 24, 24):  
            width_segment = y_pred[:, i:i+24, quantiles.index(0.9)] - y_pred[:, i:i+24, quantiles.index(0.1)]
            width_by_segment.append(np.mean(width_segment))

        lower_idx = quantiles.index(0.1)
        upper_idx = quantiles.index(0.9)
        coverage_80 = coverage_probability(y_true, y_pred, lower_idx, upper_idx)

        coverage_by_segment = []
        for i in range(0, h - 24, 24): 
            coverage_segment = coverage_probability(y_true[:, i:i+24], y_pred[:,i:i+24, :],  quantiles.index(0.1), quantiles.index(0.9))
            coverage_by_segment.append(np.mean(coverage_segment))

        results_dict = {
            "Variable": var,
            "Horizon": h,
            "RMSE": rmse,
            "CRPS": crps,
            "Winkler(10-90%)": winkler_80,
            "Coverage(10-90%)": coverage_80,
            "IntervalWidth(10-90%)": interval_width,
        }

        for j, width_val in enumerate(width_by_segment):
            results_dict[f"Width_d+{2 + j}"] = width_val
        
        for j, cov_val in enumerate(coverage_by_segment):
            results_dict[f"Coverage_d+{2 + j}"] = cov_val

        results.append(results_dict)
results_df = pd.DataFrame(results)
results_df.to_csv("C:/Users/daniq/iCloudDrive\Erasmus University Rotterdam/Master/Thesis/Output/Ablation_Effect_QuickLQRA.csv", index=False)


Generate evaluation metrics without outliers and only outliers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

def calc_rmse(y_true, y_pred, mask):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    if mask is not None:
        y_true = y_true[~mask]
        y_pred = y_pred[~mask]
    
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def pinball_loss(y_true, y_pred, quantiles):
   
    N, H, Q = y_pred.shape
    losses = np.zeros((N, H, Q))
    for q_idx, q in enumerate(quantiles):
        err = y_true - y_pred[:, :, q_idx]
        losses[:, :, q_idx] = np.maximum(q * err, (q - 1) * err)
    return losses

def calculate_pinball_loss_CRPS(y_true, Y_pred, quantiles, mask=None):
    y_true = np.array(y_true)
    Y_pred = np.array(Y_pred)
    
    if mask is not None:
        mask_3d = np.expand_dims(mask, axis=2) 
        mask_3d = np.broadcast_to(mask_3d, Y_pred.shape)  
        
        y_true = y_true.astype(float)
        y_true[mask] = np.nan
        
        Y_pred = Y_pred.astype(float)
        Y_pred[mask_3d] = np.nan

    pinball_losses = pinball_loss(y_true, Y_pred, quantiles) 

    mean_pinball_loss = np.nanmean(pinball_losses)
    crps_per_point = np.nanmean(np.trapz(pinball_losses, quantiles, axis=-1), axis=-1)
    crps = np.nanmean(crps_per_point)

    return pinball_losses, mean_pinball_loss, crps

def winkler_score(y_true, y_lower, y_upper, alpha):
    width = y_upper - y_lower
    
    under = (y_lower - y_true) * (y_true < y_lower)
    over  = (y_true - y_upper) * (y_true > y_upper)
    
    score = width + (2.0 / alpha) * (under + over)
    return score

def calculate_winkler(Y_pred, y_true, quantiles, lower_q, upper_q, mask):
    y_true = np.array(y_true)
    if mask is not None:
        mask_3d = np.expand_dims(mask, axis=2) 
        mask_3d = np.broadcast_to(mask_3d, Y_pred.shape) 

        y_true = y_true.astype(float)
        y_true[mask] = np.nan
        
        Y_pred = Y_pred.astype(float)
        Y_pred[mask_3d] = np.nan

    alpha = 1.0 - (upper_q - lower_q)
    try:
        li = quantiles.index(lower_q)
        ui = quantiles.index(upper_q)
    except ValueError:
        raise ValueError(f"lower_q={lower_q} or upper_q={upper_q} not in quantiles list")
    
    y_lower = Y_pred[:, :, li]
    y_upper = Y_pred[:, :, ui]
    y_true = np.array(y_true)
    
    scores = winkler_score(y_true, y_lower, y_upper, alpha)
    mean_score = np.nanmean(scores)
    return scores, mean_score

def load_and_prepare_quantile_data(quantile, probability_forecasts, all_quantiles):
    quantile_pred = pd.DataFrame(probability_forecasts[:, :, all_quantiles.index(quantile)])
    return quantile_pred

for Forecast_Horizon in [48, 144]: 
    results = []
    outlier_matrix = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/outlier_matrix_{Forecast_Horizon}.csv")
    outlier_mask = outlier_matrix.astype(bool).values
    #outlier_mask = ~outlier_mask # Invert mask to only focus on outliers

    true_values = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Full_true/Values_for_evaluation_{Forecast_Horizon}.csv")
    list_names = ['Arima', 'GJR_Garch', 'Garch', 'Hybrid_LSTM_lgbm', 'Hybrid_LSTM_xgb', "Hybrid_weighted_AllThree", 'Hybrid_weighted_LSTM_lgbm', 'Hybrid_weighted_LSTM_xgb', "LGBM", 'LSTM', "XGB", 'DNN']
    for model in list_names:
        y_pred = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/{model}_forecasts_{Forecast_Horizon}.csv")
        rmse = calc_rmse(true_values, y_pred, outlier_mask)
        results.append((model, rmse))
    
    prob_results = []
    all_quantiles = np.arange(0.01, 1.00, 0.01).round(2).tolist()
    model_names = ["Regular_LQRA", "Quick_LQRA", "DDNN", "DDNN_mixture"]
    for prob_model in model_names:
        prob_pred = pd.read_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Probability/{prob_model}_prob_forecasts_{Forecast_Horizon}.csv").values.reshape(290, Forecast_Horizon, 99)
        pinball_losses, avg_pinball_loss, crps = calculate_pinball_loss_CRPS(true_values, prob_pred, all_quantiles, outlier_mask)    
        scores, mean_score_1 = calculate_winkler(prob_pred, true_values, all_quantiles, 0.4, 0.6, outlier_mask)
        scores, mean_score_2 = calculate_winkler(prob_pred, true_values, all_quantiles, 0.3, 0.7, outlier_mask)
        scores, mean_score_3 = calculate_winkler(prob_pred, true_values, all_quantiles, 0.2, 0.8, outlier_mask)
        scores, mean_score_4 = calculate_winkler(prob_pred, true_values, all_quantiles, 0.1, 0.9, outlier_mask)
        scores, mean_score_5 = calculate_winkler(prob_pred, true_values, all_quantiles, 0.05, 0.95, outlier_mask)
        prob_results.append((prob_model, avg_pinball_loss, crps, mean_score_1, mean_score_2, mean_score_3, mean_score_4, mean_score_5))
        for quantile in np.arange(0.1, 1.0, 0.1).round(2):
            y_pred_quantile = load_and_prepare_quantile_data(quantile, prob_pred, all_quantiles)
            rmse = calc_rmse(true_values, y_pred_quantile, outlier_mask)
            results.append((f"{prob_model}_Q_{quantile}", rmse))
            
    results_df = pd.DataFrame(results)
    results_df.columns = ["Model", "RMSE"]
    prob_results_df = pd.DataFrame(prob_results)
    prob_results_df.columns = ["Model", "Avg_pinball_loss", "CRPS", "Winkler_Score(40%-60%)", "Winkler_Score(30%-70%)", "Winkler_Score(20%-80%)", "Winkler_Score(10%-90%)", "Winkler_Score(5%-95%)"]
    results_df.to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/Point_pred_RMSE_{Forecast_Horizon}_OnlyOutlier.csv", index=False )
    prob_results_df.to_csv(f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/Prob_pred_CRPS_Winkler_{Forecast_Horizon}_OnlyOutlier.csv", index=False)
  


Diebold-Mariano test for the point forecasts


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t
from sklearn.metrics import mean_squared_error
from statsmodels.stats.multitest import multipletests
import matplotlib.colors as mcolors
import matplotlib.patches as patches


def newey_west_variance(d, lag=None):
    T = len(d)
    d_mean = np.mean(d)
    d = d - d_mean
    if lag is None:
        lag = int(np.floor(T ** (1 / 4)))

    gamma_0 = np.sum(d * d) / T
    var = gamma_0

    for l in range(1, lag + 1):
        gamma_l = np.sum(d[l:] * d[:-l]) / T
        weight = 1 - l / (lag + 1)
        var += 2 * weight * gamma_l

    return var

def diebold_mariano_test(y_true, y_pred1, y_pred2, h=1, power=2):
    y_true = y_true.flatten()
    y_pred1 = y_pred1.flatten()
    y_pred2 = y_pred2.flatten()

    e1 = y_true - y_pred1
    e2 = y_true - y_pred2
    d = np.abs(e1)**power - np.abs(e2)**power

    d_mean = np.mean(d)
    N = len(d)
    var_d = newey_west_variance(d, lag=h - 1 if h > 1 else 0)

    dm_stat = d_mean / np.sqrt(var_d / N)
    p_value = t.cdf(dm_stat, df=N - 1) 

    return dm_stat, p_value


for Forecast_Horizon in [48, 144]:
    print(f"\n--- Forecast Horizon: {Forecast_Horizon} ---")
    model_preds = {}
    true_values = pd.read_csv(
        f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Full_true/Values_for_evaluation_{Forecast_Horizon}.csv"
    ).values

    list_names_point = ['Arima', 'GJR_Garch', 'Garch','LSTM', 'LGBM', 'XGB',
                        'Hybrid_LSTM_lgbm', 'Hybrid_LSTM_xgb', 'Hybrid_weighted_AllThree',
                        'Hybrid_weighted_LSTM_lgbm', 'Hybrid_weighted_LSTM_xgb', 'DNN']

    for model in list_names_point:
        y_pred = pd.read_csv(
            f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Point/{model}_forecasts_{Forecast_Horizon}.csv"
        ).values
        model_preds[model] = y_pred

    list_names_prob = ["DDNN", "DDNN_mixture", "Regular_LQRA", "Quick_LQRA"]

    for model in list_names_prob:
        y_pred_quantiles = pd.read_csv(
            f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Probability/{model}_prob_forecasts_{Forecast_Horizon}.csv"
        ).values.reshape(290, Forecast_Horizon, 99)
        
        median_forecast = y_pred_quantiles[:, :, 49]
        model_preds[f"{model}_median"] = median_forecast
    combined_models = list_names_point + [f"{m}_median" for m in list_names_prob]

    dm_stat_matrix = pd.DataFrame(index=combined_models, columns=combined_models)
    p_val_matrix = pd.DataFrame(index=combined_models, columns=combined_models)

    for i, model_i in enumerate(combined_models):
        for j, model_j in enumerate(combined_models):
            if i == j:
                dm_stat_matrix.loc[model_i, model_j] = 0.0
                p_val_matrix.loc[model_i, model_j] = 1.0
            else:
                dm_stat, p_val = diebold_mariano_test(
                    true_values, model_preds[model_i], model_preds[model_j], h=Forecast_Horizon
                )
                dm_stat_matrix.loc[model_i, model_j] = dm_stat
                p_val_matrix.loc[model_i, model_j] = p_val

    dm_stat_float = dm_stat_matrix.astype(float)
    p_val_float = p_val_matrix.astype(float)

    mask_not_sig = (p_val_float >= 0.1) | (dm_stat_float >= 0) 

    filtered_matrix = p_val_float.mask(mask_not_sig)

    annot_matrix = filtered_matrix.copy().astype(str)
    for i in range(len(annot_matrix)):
        for j in range(len(annot_matrix)):
            if i == j:
                annot_matrix.iloc[i, j] = 'X'
            elif pd.isna(filtered_matrix.iloc[i, j]):
                annot_matrix.iloc[i, j] = ''
            else:
                annot_matrix.iloc[i, j] = f"{filtered_matrix.iloc[i, j]:.3f}"

    plt.figure(figsize=(8, 6))
    gray_color = '#444444'
    mask = filtered_matrix.isnull()
    mask.values[np.diag_indices_from(mask)] = False
    fig, ax = plt.subplots(figsize=(10, 8))
    for (i, j), val in np.ndenumerate(mask.values):
        if val or i == j:
            ax.add_patch(patches.Rectangle((j, i), 1, 1, fill=True, color=gray_color, lw=0))
    sns.heatmap(
        filtered_matrix,
        annot=False,
        cmap='coolwarm_r',
        vmin=0, vmax=0.1,
        cbar_kws={'label': 'p-value'},
        linewidths=0,
        linecolor='none',
        mask=mask,
        ax=ax
    )
    n = filtered_matrix.shape[0]
    for i in range(n + 1):
        ax.axhline(i, color='white', linewidth=0.3)
        ax.axvline(i, color='white', linewidth=0.3)
    for i in range(n):
        ax.plot(i + 0.5, i + 0.5, marker='x', markersize=14, color='white', markeredgewidth=2)
    plt.xticks(rotation=45, ha="right")
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig(
        f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/DM_pval_heatmap_{Forecast_Horizon}.png"
    )
    plt.show()

Diebold-Mariano test for the probabilistic forecasts

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t
import matplotlib.patches as patches
import os


def pinball_loss(y_true, y_pred, quantiles):
    N, H, Q = y_pred.shape
    losses = np.zeros((N, H, Q))
    for q_idx, q in enumerate(quantiles):
        err = y_true - y_pred[:, :, q_idx]
        losses[:, :, q_idx] = np.maximum(q * err, (q - 1) * err)
    return losses

def calculate_pinball_loss_CRPS(y_true, Y_pred, quantiles):
    pinball_losses = pinball_loss(np.array(y_true), Y_pred, np.array(quantiles))
    crps = np.mean(np.trapz(pinball_losses, np.array(quantiles), axis=-1))
    return pinball_losses, np.mean(pinball_losses), crps

def newey_west_variance(d, lag=None):
    T = len(d)
    d_mean = np.mean(d)
    d = d - d_mean
    if lag is None:
        lag = int(np.floor(T ** (1 / 4)))

    gamma_0 = np.sum(d * d) / T
    var = gamma_0

    for l in range(1, lag + 1):
        gamma_l = np.sum(d[l:] * d[:-l]) / T
        weight = 1 - l / (lag + 1)
        var += 2 * weight * gamma_l

    return var


def diebold_mariano_test(pl1, pl2, h=1):
    d = pl1 - pl2
    d_mean = np.mean(d)
    N = len(d)
    var_d = newey_west_variance(d, lag=h - 1 if h > 1 else 0)
    dm_stat = d_mean / np.sqrt(var_d / N)
    p_value = t.cdf(dm_stat, df=N - 1)  
    return dm_stat, p_value




for Forecast_Horizon in [48, 144]:
    true_values = pd.read_csv(
        f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Full_true/Values_for_evaluation_{Forecast_Horizon}.csv"
    ).values

    list_names = ["DDNN", "DDNN_mixture", "Regular_LQRA", "Very_quick_LQRA"]
    model_losses = {}

    for model in list_names:
        y_pred = pd.read_csv(
                f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Forecasts/Probability/{model}_prob_forecasts_{Forecast_Horizon}.csv"
        ).values.reshape(290, Forecast_Horizon, 99)
        pinball, _, _ = calculate_pinball_loss_CRPS(true_values, y_pred, np.arange(0.01, 1.00, 0.01))
        daily_mean_loss = pinball.mean(axis=(1, 2))  
        model_losses[model] = daily_mean_loss

    dm_stat_matrix = pd.DataFrame(index=list_names, columns=list_names)
    p_val_matrix = pd.DataFrame(index=list_names, columns=list_names)

    for i, model_i in enumerate(list_names):
        for j, model_j in enumerate(list_names):
            if i == j:
                dm_stat_matrix.loc[model_i, model_j] = 0.0
                p_val_matrix.loc[model_i, model_j] = 1.0
            else:
                dm_stat, p_val = diebold_mariano_test(
                    model_losses[model_i], model_losses[model_j], h=Forecast_Horizon
                )
                dm_stat_matrix.loc[model_i, model_j] = dm_stat
                p_val_matrix.loc[model_i, model_j] = p_val


    dm_stat_float = dm_stat_matrix.astype(float)
    p_val_float = p_val_matrix.astype(float)

    mask_not_sig = (p_val_float >= 0.1) | (dm_stat_float >= 0)
    filtered_matrix = p_val_float.mask(mask_not_sig)

    annot_matrix = filtered_matrix.copy().astype(str)
    for i in range(len(annot_matrix)):
        for j in range(len(annot_matrix)):
            if i == j:
                annot_matrix.iloc[i, j] = 'X'
            elif pd.isna(filtered_matrix.iloc[i, j]):
                annot_matrix.iloc[i, j] = ''
            else:
                annot_matrix.iloc[i, j] = f"{filtered_matrix.iloc[i, j]:.3f}"

    plt.figure(figsize=(5, 4))
    gray_color = '#444444'
    mask = filtered_matrix.isnull()
    mask.values[np.diag_indices_from(mask)] = False

    fig, ax = plt.subplots(figsize=(8, 6))

    for (i, j), val in np.ndenumerate(mask.values):
        if val or i == j:
            ax.add_patch(
                patches.Rectangle(
                    (j, i), 1, 1,
                    fill=True,
                    color=gray_color,
                    lw=0
                )
            )

    sns.heatmap(
        filtered_matrix,
        annot=False,
        cmap='coolwarm_r',
        vmin=0, vmax=0.1,
        cbar_kws={'label': 'p-value'},
        linewidths=0,
        linecolor='none',
        mask=mask,
        ax=ax
    )

    n = filtered_matrix.shape[0]
    for i in range(n + 1):
        ax.axhline(i, color='white', linewidth=0.3)
        ax.axvline(i, color='white', linewidth=0.3)

    for i in range(n):
        ax.plot(i + 0.5, i + 0.5, marker='x', markersize=34, color='white', markeredgewidth=2)

    plt.xticks(rotation=45, ha="right")
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig(
    f"C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Evaluation_Metrics/DM_prob_pval_heatmap_{Forecast_Horizon}.png"
    )
    plt.show()
